# **Chapter 10: Introduction to Feature Engineering**

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Understand what features represent in the context of time-series prediction systems
- Identify different types of features applicable to financial time-series data, specifically for the NEPSE (Nepal Stock Exchange) stock prediction system
- Apply domain knowledge from financial markets to engineer meaningful features from raw OHLCV (Open, High, Low, Close, Volume) data
- Distinguish between feature engineering and feature selection processes
- Recognize common pitfalls such as look-ahead bias and data leakage when working with stock market data
- Document features effectively for reproducibility and team collaboration

---

## **10.1 What Are Features?**

In machine learning, **features** (also known as variables, attributes, or predictors) are the measurable properties or characteristics of the phenomenon being observed. In the context of our NEPSE stock prediction system, features are the quantifiable attributes derived from historical market data that help the model learn patterns and predict future price movements.

Raw data from the NEPSE CSV format contains columns like `Open`, `High`, `Low`, `Close`, `Volume`, `VWAP`, etc. While these raw values can serve as basic features, feature engineering transforms these raw inputs into more informative representations that capture temporal dynamics, statistical properties, and domain-specific patterns.

```python
import pandas as pd
import numpy as np

# Load NEPSE stock data
# CSV Structure: S.No,Symbol,Conf.,Open,High,Low,Close,LTP,Close - LTP,
# Close - LTP %,VWAP,Vol,Prev. Close,Turnover,Trans.,Diff,Range,Diff %,
# Range %,VWAP %,52 Weeks High,52 Weeks Low

df = pd.read_csv('nepse_data.csv')

# Display basic information about the raw features
print("Raw Feature Columns:", df.columns.tolist())
print("\nFirst 5 rows of raw features:")
print(df[['Symbol', 'Open', 'High', 'Low', 'Close', 'Vol']].head())

# Basic statistical features from raw data
print("\nBasic Statistics of Close Price:")
print(df['Close'].describe())

# Creating a simple feature: Daily Return
# This is a derived feature showing percentage change from previous close
df['Daily_Return'] = ((df['Close'] - df['Prev. Close']) / df['Prev. Close']) * 100

print("\nDaily Return Feature (First 5 values):")
print(df[['Symbol', 'Close', 'Prev. Close', 'Daily_Return']].head())
```

**Explanation:**

The code above demonstrates the fundamental concept of features using the NEPSE dataset. 

First, we load the CSV data containing the raw market data. The columns `Open`, `High`, `Low`, `Close`, and `Vol` represent the most basic features—raw price and volume data points recorded during the trading session.

The `df['Close'].describe()` operation generates statistical features (mean, standard deviation, min, max) that characterize the distribution of closing prices. These statistical summaries serve as aggregate features that capture the central tendency and volatility of the stock.

Finally, we engineer a new feature called `Daily_Return` by calculating the percentage change between the current `Close` and `Prev. Close`. This derived feature is crucial for time-series prediction because it normalizes price changes across different stocks and time periods, making patterns more comparable and learnable by machine learning models. The daily return captures the momentum and direction of price movement, which is often more predictive than absolute price levels.

---

## **10.2 The Feature Engineering Process**

Feature engineering is the systematic process of transforming raw data into features that better represent the underlying problem to the predictive models, resulting in improved model accuracy. For the NEPSE prediction system, this process involves a structured pipeline that converts raw OHLCV data into a rich feature set suitable for machine learning algorithms.

The process typically follows these stages: **Data Understanding** (exploring the NEPSE CSV structure), **Data Cleaning** (handling missing values in volume or price data), **Feature Construction** (creating lag features, technical indicators), **Feature Transformation** (normalization, scaling), and **Feature Validation** (checking for data leakage and stationarity).

```python
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt

class NEPSEFeatureEngineer:
    """
    A comprehensive feature engineering pipeline for NEPSE stock data.
    This class encapsulates the entire feature engineering process.
    """
    
    def __init__(self, data_path):
        self.data_path = data_path
        self.raw_data = None
        self.engineered_data = None
        self.scaler = StandardScaler()
        
    def load_data(self):
        """Stage 1: Data Loading and Initial Understanding"""
        self.raw_data = pd.read_csv(self.data_path)
        
        # Convert date columns if present, otherwise create sequence
        if 'Date' in self.raw_data.columns:
            self.raw_data['Date'] = pd.to_datetime(self.raw_data['Date'])
            self.raw_data = self.raw_data.sort_values('Date')
        else:
            # If no date column, assume data is ordered by S.No
            self.raw_data = self.raw_data.sort_values('S.No')
            
        print(f"Data loaded: {len(self.raw_data)} records")
        print(f"Columns: {self.raw_data.columns.tolist()}")
        return self
    
    def clean_data(self):
        """Stage 2: Data Cleaning and Preprocessing"""
        # Handle missing values in critical columns
        critical_columns = ['Open', 'High', 'Low', 'Close', 'Vol']
        
        for col in critical_columns:
            if col in self.raw_data.columns:
                # Forward fill for missing values (carry last known value)
                self.raw_data[col] = self.raw_data[col].fillna(method='ffill')
                # Backward fill for any remaining NaNs at the beginning
                self.raw_data[col] = self.raw_data[col].fillna(method='bfill')
        
        # Remove outliers (prices that are unrealistic)
        # Using IQR method for Close price
        Q1 = self.raw_data['Close'].quantile(0.25)
        Q3 = self.raw_data['Close'].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Filter out extreme outliers
        self.raw_data = self.raw_data[
            (self.raw_data['Close'] >= lower_bound) & 
            (self.raw_data['Close'] <= upper_bound)
        ]
        
        print(f"Data cleaned: {len(self.raw_data)} records remaining")
        return self
    
    def construct_features(self):
        """Stage 3: Feature Construction"""
        df = self.raw_data.copy()
        
        # 3.1 Lag Features (Previous values)
        # These capture temporal dependencies
        for lag in [1, 3, 5]:
            df[f'Close_Lag_{lag}'] = df['Close'].shift(lag)
            df[f'Volume_Lag_{lag}'] = df['Vol'].shift(lag)
        
        # 3.2 Rolling Window Features (Moving statistics)
        # These capture trends and volatility over time
        windows = [5, 10, 20]
        for window in windows:
            # Moving averages (trend indicators)
            df[f'SMA_{window}'] = df['Close'].rolling(window=window).mean()
            df[f'EMA_{window}'] = df['Close'].ewm(span=window).mean()
            
            # Volatility indicators
            df[f'Std_{window}'] = df['Close'].rolling(window=window).std()
            df[f'Range_{window}'] = (
                df['High'].rolling(window=window).max() - 
                df['Low'].rolling(window=window).min()
            )
            
            # Volume indicators
            df[f'Vol_SMA_{window}'] = df['Vol'].rolling(window=window).mean()
        
        # 3.3 Technical Indicators (Domain-specific features)
        # Relative Strength Index (RSI)
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))
        
        # MACD (Moving Average Convergence Divergence)
        ema_12 = df['Close'].ewm(span=12).mean()
        ema_26 = df['Close'].ewm(span=26).mean()
        df['MACD'] = ema_12 - ema_26
        df['MACD_Signal'] = df['MACD'].ewm(span=9).mean()
        
        # 3.4 Time-based Features
        if 'Date' in df.columns:
            df['Day_of_Week'] = df['Date'].dt.dayofweek
            df['Month'] = df['Date'].dt.month
            df['Quarter'] = df['Date'].dt.quarter
            df['Is_Month_Start'] = df['Date'].dt.is_month_start.astype(int)
            df['Is_Month_End'] = df['Date'].dt.is_month_end.astype(int)
        
        # 3.5 Price-based Features
        df['Daily_Return'] = df['Close'].pct_change()
        df['Price_Range'] = df['High'] - df['Low']
        df['Price_Change'] = df['Close'] - df['Open']
        df['Upper_Shadow'] = df['High'] - df[['Close', 'Open']].max(axis=1)
        df['Lower_Shadow'] = df[['Close', 'Open']].min(axis=1) - df['Low']
        
        # Remove rows with NaN values created by lag/rolling operations
        df = df.dropna()
        
        self.engineered_data = df
        print(f"Features constructed: {len(df.columns)} features")
        print(f"Feature columns: {df.columns.tolist()}")
        return self
    
    def transform_features(self):
        """Stage 4: Feature Transformation and Scaling"""
        # Separate features and target
        feature_cols = [col for col in self.engineered_data.columns 
                       if col not in ['Close', 'Symbol', 'Date', 'S.No', 'Conf.']]
        
        X = self.engineered_data[feature_cols]
        y = self.engineered_data['Close']
        
        # Scale features
        X_scaled = self.scaler.fit_transform(X)
        X_scaled_df = pd.DataFrame(X_scaled, columns=feature_cols, index=X.index)
        
        print(f"Features transformed and scaled")
        print(f"Feature matrix shape: {X_scaled_df.shape}")
        return X_scaled_df, y
    
    def validate_features(self):
        """Stage 5: Feature Validation"""
        # Check for data leakage (features that use future information)
        # In a real system, this would be more sophisticated
        
        # Check for stationarity (important for time-series)
        from scipy import stats
        
        close_prices = self.engineered_data['Close']
        adf_result = stats.adfuller(close_prices)
        
        print(f"ADF Statistic: {adf_result[0]}")
        print(f"p-value: {adf_result[1]}")
        if adf_result[1] <= 0.05:
            print("Series is stationary")
        else:
            print("Series is non-stationary (consider differencing)")
        
        # Check for multicollinearity
        corr_matrix = self.engineered_data[['Close', 'SMA_5', 'SMA_10', 'RSI']].corr()
        print("\nCorrelation Matrix:")
        print(corr_matrix)
        
        return self

# Usage example
if __name__ == "__main__":
    # Initialize the feature engineering pipeline
    pipeline = NEPSEFeatureEngineer('nepse_data.csv')
    
    # Execute the pipeline stages
    (pipeline
     .load_data()
     .clean_data()
     .construct_features()
     .validate_features())
    
    # Get transformed features for modeling
    X, y = pipeline.transform_features()
```

**Explanation:**

This comprehensive example demonstrates the complete feature engineering process for the NEPSE stock prediction system through a structured pipeline class.

**Stage 1: Data Loading and Understanding**
The `load_data()` method initializes the pipeline by loading the NEPSE CSV data. It handles the specific structure of NEPSE data, sorting by `S.No` (serial number) if no date column exists, or by `Date` if available. This ensures temporal ordering is maintained, which is critical for time-series feature engineering.

**Stage 2: Data Cleaning**
The `clean_data()` method addresses data quality issues specific to financial time-series. It uses forward-fill (`ffill`) and backward-fill (`bfill`) methods to handle missing values, which is appropriate for stock data where the last known price is a reasonable estimate for brief gaps. It also implements outlier detection using the Interquartile Range (IQR) method to remove erroneous price spikes that could distort feature calculations.

**Stage 3: Feature Construction**
This is the core stage where raw NEPSE data is transformed into predictive features:

- **Lag Features**: The code creates `Close_Lag_1`, `Close_Lag_3`, and `Close_Lag_5`, which represent the closing price 1, 3, and 5 periods ago. These capture temporal dependencies and autocorrelation in stock prices.

- **Rolling Window Features**: Simple Moving Averages (`SMA_5`, `SMA_10`, `SMA_20`) and Exponential Moving Averages (`EMA`) are calculated to identify trends. Standard deviation (`Std`) and range features capture volatility over different time horizons.

- **Technical Indicators**: The Relative Strength Index (RSI) measures momentum and overbought/oversold conditions. MACD (Moving Average Convergence Divergence) identifies trend changes and momentum shifts. These are domain-specific features derived from financial technical analysis.

- **Time-Based Features**: Day of week, month, quarter, and month-start/end indicators capture calendar effects and seasonal patterns in trading behavior.

- **Price-Based Features**: Daily returns, price ranges, upper/lower shadows (candlestick patterns), and price changes provide normalized measures of price action.

**Stage 4: Feature Transformation**
The `transform_features()` method applies `StandardScaler` to normalize feature distributions. This is essential because features like volume (in millions) and RSI (0-100) operate on vastly different scales. Standardization ensures that the model treats all features equally during training.

**Stage 5: Feature Validation**
The `validate_features()` method checks for stationarity using the Augmented Dickey-Fuller (ADF) test, which is crucial for time-series models. It also examines the correlation matrix to detect multicollinearity between features (e.g., high correlation between `Close` and `SMA_5`), which could cause instability in linear models.

This structured pipeline ensures that the NEPSE stock prediction system receives high-quality, informative features that capture the complex temporal dynamics of the Nepalese stock market while avoiding common pitfalls like data leakage and look-ahead bias.

---

## **10.4 Domain Knowledge Integration**

Domain knowledge integration is the process of incorporating expert understanding of a specific field into the feature engineering pipeline. For the NEPSE stock prediction system, this means leveraging financial market theories, trading practices, and economic principles specific to the Nepalese stock market to create features that capture meaningful market behaviors rather than just statistical patterns.

The Nepalese stock market has unique characteristics: it operates Sunday through Thursday (unlike Western markets), has specific regulatory constraints, experiences high volatility due to limited liquidity, and shows strong seasonal patterns around fiscal year endings (mid-July). Understanding these nuances allows us to engineer features that a generic time-series approach might miss.

```python
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

class NEPSEDomainFeatureEngineer:
    """
    Feature engineering class that integrates Nepalese stock market domain knowledge.
    NEPSE-specific characteristics:
    - Trading days: Sunday to Thursday (5 days a week)
    - High volatility, low liquidity compared to major exchanges
    - Strong fiscal year effects (fiscal year ends mid-July)
    - Circuit breakers: 4% (first), 5% (second), 6% (third) daily limits
    """
    
    def __init__(self, df):
        self.df = df.copy()
        self._ensure_datetime()
        
    def _ensure_datetime(self):
        """Ensure Date column is datetime type"""
        if 'Date' in self.df.columns:
            self.df['Date'] = pd.to_datetime(self.df['Date'])
            self.df = self.df.sort_values('Date')
    
    def add_nepse_trading_calendar_features(self):
        """
        NEPSE trades Sunday-Thursday.
        These features capture trading day effects specific to Nepalese market.
        """
        df = self.df
        
        # Day of week (0=Sunday, 4=Thursday in NEPSE context)
        # Note: In pandas, Monday=0, Sunday=6, so we adjust
        df['Day_of_Week'] = df['Date'].dt.dayofweek
        df['Is_Sunday'] = (df['Day_of_Week'] == 6).astype(int)  # Sunday often has different volume
        df['Is_Thursday'] = (df['Day_of_Week'] == 4).astype(int)  # Thursday is last trading day
        
        # Weekend gap (days since last trading day)
        # In NEPSE, Friday and Saturday are weekend
        df['Days_Since_Last_Trade'] = df['Date'].diff().dt.days
        
        # Trading week of month (NEPSE has ~4 trading weeks per month)
        df['Trading_Week'] = (df['Date'].dt.day - 1) // 7 + 1
        
        self.df = df
        return self
    
    def add_fiscal_year_features(self):
        """
        Nepal's fiscal year runs mid-July to mid-July (roughly July 16 - July 15).
        This creates unique seasonal patterns in stock trading due to:
        - Year-end portfolio adjustments
        - Tax considerations
        - Quarterly reporting cycles
        """
        df = self.df
        
        # Nepali fiscal year (approximate mapping using Gregorian calendar)
        # FY starts ~July 16
        month = df['Date'].dt.month
        day = df['Date'].dt.day
        
        # Determine fiscal year (year starts in July)
        fiscal_year = df['Date'].dt.year
        # If before July 16, it's part of previous fiscal year
        mask = (month < 7) | ((month == 7) & (day < 16))
        fiscal_year[mask] = fiscal_year[mask] - 1
        
        df['Fiscal_Year'] = fiscal_year
        
        # Fiscal quarter (Q1: Jul-Sep, Q2: Oct-Dec, Q3: Jan-Mar, Q4: Apr-Jul)
        quarter_mapping = {7: 1, 8: 1, 9: 1, 
                          10: 2, 11: 2, 12: 2,
                          1: 3, 2: 3, 3: 3,
                          4: 4, 5: 4, 6: 4, 7: 4}
        
        # Adjust for fiscal year quarters
        df['Fiscal_Quarter'] = month.map(lambda m: 
            1 if m in [7, 8, 9] else
            2 if m in [10, 11, 12] else
            3 if m in [1, 2, 3] else 4
        )
        
        # Days to fiscal year end (important for tax-loss harvesting patterns)
        # FY ends ~July 15
        year_end = pd.to_datetime(df['Date'].dt.year.astype(str) + '-07-15')
        # Adjust for dates after July 15 (belong to next FY end)
        year_end[df['Date'].dt.month >= 7] = pd.to_datetime(
            (df['Date'].dt.year + 1).astype(str) + '-07-15'
        )
        
        df['Days_to_FY_End'] = (year_end - df['Date']).dt.days
        
        self.df = df
        return self
    
    def add_circuit_breaker_features(self):
        """
        NEPSE has circuit breakers at 4%, 5%, and 6% daily price limits.
        These features identify when stocks are hitting these limits,
        which indicates extreme sentiment and potential reversal points.
        """
        df = self.df
        
        # Calculate daily price change percentage
        df['Daily_Change_Pct'] = ((df['Close'] - df['Prev. Close']) / df['Prev. Close']) * 100
        
        # Circuit breaker levels (NEPSE specific)
        df['Hit_Upper_Circuit'] = (df['Daily_Change_Pct'] >= 4).astype(int)
        df['Hit_Lower_Circuit'] = (df['Daily_Change_Pct'] <= -4).astype(int)
        
        # Proximity to circuit breakers (how close to 4% limit)
        df['Upper_Circuit_Distance'] = 4 - df['Daily_Change_Pct']
        df['Lower_Circuit_Distance'] = df['Daily_Change_Pct'] - (-4)
        
        # Volatility regime (based on how often circuit breakers are hit)
        df['Circuit_Breaker_Regime'] = df['Hit_Upper_Circuit'].rolling(window=20).sum() - \
                                       df['Hit_Lower_Circuit'].rolling(window=20).sum()
        
        self.df = df
        return self
    
    def get_feature_summary(self):
        """Return summary of engineered features"""
        feature_categories = {
            'Raw_Price': ['Open', 'High', 'Low', 'Close', 'LTP'],
            'Volume': ['Vol', 'Turnover', 'Trans.'],
            'Calendar': ['Day_of_Week', 'Is_Sunday', 'Is_Thursday', 'Fiscal_Year', 'Fiscal_Quarter'],
            'Lag': [col for col in self.df.columns if 'Lag_' in col],
            'Rolling': [col for col in self.df.columns if 'SMA_' in col or 'EMA_' in col or 'Std_' in col],
            'Technical': ['RSI', 'MACD', 'MACD_Signal'],
            'Circuit_Breaker': ['Daily_Change_Pct', 'Hit_Upper_Circuit', 'Circuit_Breaker_Regime']
        }
        
        summary = {}
        for category, features in feature_categories.items():
            available = [f for f in features if f in self.df.columns]
            if available:
                summary[category] = len(available)
        
        return summary

# Usage Example
if __name__ == "__main__":
    # Initialize the feature engineering pipeline
    engineer = NEPSEFeatureEngineer('nepse_data.csv')
    
    # Execute the complete pipeline
    (engineer
     .load_data()
     .clean_data()
     .add_nepse_trading_calendar_features()
     .add_fiscal_year_features()
     .add_circuit_breaker_features()
     .construct_features())
    
    # Get feature summary
    summary = engineer.get_feature_summary()
    print("\nFeature Engineering Summary:")
    for category, count in summary.items():
        print(f"  {category}: {count} features")
    
    # Display sample of engineered features
    print("\nSample of Engineered Features:")
    feature_cols = [col for col in engineer.df.columns 
                   if col not in ['S.No', 'Symbol', 'Conf.']]
    print(engineer.df[feature_cols[:10]].head())
```

**Explanation:**

This comprehensive example demonstrates the feature engineering process specifically tailored for the NEPSE stock prediction system, integrating domain knowledge unique to the Nepalese stock market.

**The Pipeline Architecture:**
The `NEPSEFeatureEngineer` class encapsulates the entire feature engineering workflow as a fluent interface (method chaining), allowing stages to be executed sequentially. This design pattern ensures reproducibility and makes the pipeline easy to modify or extend.

**Stage 1: Data Loading and Understanding**
The `load_data()` method handles the specific NEPSE CSV structure. It recognizes that NEPSE data may or may not have explicit Date columns, falling back to `S.No` (serial number) for temporal ordering. This is crucial because NEPSE data exports sometimes lack proper timestamps, relying instead on sequential ordering.

**Stage 2: Data Cleaning**
The `clean_data()` method addresses real-world data quality issues in NEPSE feeds. It uses forward-fill and backward-fill for missing values, which is appropriate for financial time-series where the last traded price remains valid until the next trade. It also implements the Interquartile Range (IQR) method to remove extreme outliers that might represent data entry errors or flash crashes not representative of normal market conditions.

**Stage 3: Domain-Specific Feature Engineering**
This is where deep NEPSE domain knowledge is integrated:

*Trading Calendar Features*: Unlike major international exchanges that trade Monday-Friday, NEPSE operates Sunday through Thursday. The code creates binary indicators `Is_Sunday` and `Is_Thursday` because these days often show different trading patterns (Sunday being the first day after the weekend gap, Thursday being the pre-weekend close). The `Days_Since_Last_Trade` feature accounts for the Friday-Saturday weekend gap unique to NEPSE.

*Fiscal Year Features*: Nepal's fiscal year runs from mid-July to mid-July (Shrawan to Ashad in the Nepali calendar). This creates distinct seasonal patterns in stock trading due to fiscal year-end portfolio adjustments, tax-loss harvesting, and quarterly earnings cycles aligned with this calendar. The code calculates `Fiscal_Year`, `Fiscal_Quarter`, and `Days_to_FY_End` to capture these seasonal effects.

*Circuit Breaker Features*: NEPSE implements circuit breakers at 4%, 5%, and 6% daily price movement limits. When a stock hits these limits, it indicates extreme sentiment and potential reversal points. The code creates binary indicators for upper/lower circuit hits and calculates proximity to these limits, which serve as regime indicators for volatility clustering.

**Stage 4: Feature Transformation**
After construction, features are standardized using `StandardScaler` to ensure all features contribute equally to the model regardless of their original scales (e.g., volume in millions vs. RSI in 0-100).

**Stage 5: Validation**
The pipeline includes validation checks for stationarity using the Augmented Dickey-Fuller test (essential for time-series models) and correlation analysis to detect multicollinearity between derived features.

This domain-integrated approach ensures that the NEPSE prediction model leverages not just statistical patterns but the specific structural characteristics of the Nepalese stock market, from its unique trading calendar to its regulatory circuit breaker mechanisms.

---

## **10.3 Feature Types in Time-Series**

Time-series feature engineering requires specialized approaches that account for temporal dependencies, sequential patterns, and the non-stationary nature of financial data. Unlike cross-sectional data where observations are independent, time-series features must respect the arrow of time and capture the evolving statistical properties of the series.

For the NEPSE prediction system, we categorize features into several distinct types, each capturing different aspects of market behavior: **Lag Features** (historical values), **Rolling Window Features** (moving statistics), **Expanding Window Features** (cumulative statistics), **Date/Time Features** (temporal context), **Domain-Specific Indicators** (technical analysis), and **Interaction Features** (combinations of raw features).

```python
import pandas as pd
import numpy as np
import talib  # Technical Analysis Library
from scipy.stats import skew, kurtosis
from typing import List, Dict, Tuple

class NEPSETimeSeriesFeatures:
    """
    Comprehensive time-series feature engineering for NEPSE stock data.
    Implements various feature types specific to financial time-series.
    """
    
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        self.feature_metadata = {}
        
    def create_lag_features(self, columns: List[str], lags: List[int]) -> 'NEPSETimeSeriesFeatures':
        """
        Create lag features (autoregressive features).
        
        Lag features are the most fundamental time-series features, representing
        the value of a variable at previous time steps. They capture temporal
        dependencies and autocorrelation structures.
        
        For NEPSE: Lag-1 of Close price represents yesterday's closing price,
        which is often the strongest predictor of today's price.
        """
        for col in columns:
            if col not in self.df.columns:
                continue
                
            for lag in lags:
                feature_name = f'{col}_Lag_{lag}'
                self.df[feature_name] = self.df[col].shift(lag)
                
                # Store metadata
                self.feature_metadata[feature_name] = {
                    'type': 'lag',
                    'source_column': col,
                    'lag_periods': lag,
                    'description': f'{col} value from {lag} periods ago'
                }
        
        print(f"Created {len(lags) * len(columns)} lag features")
        return self
    
    def create_rolling_window_features(self, columns: List[str], 
                                     windows: List[int]) -> 'NEPSETimeSeriesFeatures':
        """
        Create rolling window (moving) statistics.
        
        Rolling window features calculate statistics over a fixed window of
        recent observations. They capture local trends, volatility, and
        momentum while adapting to changing market conditions.
        
        For NEPSE: A 20-day rolling average represents the monthly trend
        (since NEPSE trades ~20 days per month).
        """
        for col in columns:
            if col not in self.df.columns:
                continue
                
            for window in windows:
                # Statistical measures
                self.df[f'{col}_SMA_{window}'] = self.df[col].rolling(window=window).mean()
                self.df[f'{col}_EMA_{window}'] = self.df[col].ewm(span=window, adjust=False).mean()
                self.df[f'{col}_Std_{window}'] = self.df[col].rolling(window=window).std()
                self.df[f'{col}_Min_{window}'] = self.df[col].rolling(window=window).min()
                self.df[f'{col}_Max_{window}'] = self.df[col].rolling(window=window).max()
                self.df[f'{col}_Range_{window}'] = (
                    self.df[f'{col}_Max_{window}'] - self.df[f'{col}_Min_{window}']
                )
                
                # Higher-order statistics
                self.df[f'{col}_Skew_{window}'] = self.df[col].rolling(window=window).skew()
                self.df[f'{col}_Kurt_{window}'] = self.df[col].rolling(window=window).kurt()
                
                # Technical indicators derived from rolling windows
                # Rate of Change (ROC)
                self.df[f'{col}_ROC_{window}'] = (
                    (self.df[col] - self.df[col].shift(window)) / 
                    self.df[col].shift(window) * 100
                )
                
        print(f"Created rolling window features for windows: {windows}")
        return self
    
    def create_expanding_window_features(self, columns: List[str]) -> 'NEPSETimeSeriesFeatures':
        """
        Create expanding window (cumulative) statistics.
        
        Expanding window features calculate statistics from the beginning of
        the series up to the current point. They capture long-term trends
        and historical context that rolling windows might miss.
        
        For NEPSE: Expanding mean shows the all-time average price trend,
        useful for identifying long-term support/resistance levels.
        """
        for col in columns:
            if col not in self.df.columns:
                continue
            
            # Cumulative statistics
            self.df[f'{col}_Exp_Mean'] = self.df[col].expanding().mean()
            self.df[f'{col}_Exp_Std'] = self.df[col].expanding().std()
            self.df[f'{col}_Exp_Min'] = self.df[col].expanding().min()
            self.df[f'{col}_Exp_Max'] = self.df[col].expanding().max()
            
            # Cumulative returns (since inception)
            self.df[f'{col}_Cumulative_Return'] = (
                (self.df[col] / self.df[col].iloc[0]) - 1
            ) * 100
            
            # Running count of observations
            self.df[f'{col}_Count'] = self.df[col].expanding().count()
            
        print("Created expanding window features")
        return self
    
    def create_datetime_features(self) -> 'NEPSETimeSeriesFeatures':
        """
        Create time-based features specific to NEPSE trading patterns.
        
        Temporal features capture cyclical patterns, seasonality, and
        calendar effects that influence stock prices.
        """
        df = self.df
        
        if 'Date' not in df.columns:
            print("No Date column found, skipping datetime features")
            return self
        
        # Basic temporal features
        df['Year'] = df['Date'].dt.year
        df['Month'] = df['Date'].dt.month
        df['Day'] = df['Date'].dt.day
        df['Day_of_Week'] = df['Date'].dt.dayofweek  # 0=Monday, 6=Sunday
        df['Quarter'] = df['Date'].dt.quarter
        df['Day_of_Year'] = df['Date'].dt.dayofyear
        
        # NEPSE-specific trading patterns
        # NEPSE trades Sunday-Thursday (Friday-Saturday weekend)
        df['Is_Weekend_Proximity'] = ((df['Day_of_Week'] == 4) |  # Thursday (last trading day)
                                       (df['Day_of_Week'] == 6)).astype(int)  # Sunday (first trading day)
        
        # Month-end effects (common in emerging markets)
        df['Is_Month_End'] = (df['Date'].dt.is_month_end).astype(int)
        df['Days_to_Month_End'] = (df['Date'] + pd.offsets.MonthEnd(0) - df['Date']).dt.days
        
        # Fiscal year effects (Nepal: mid-July to mid-July)
        # Q4 (April-July) often shows different patterns due to year-end accounting
        df['Is_Fiscal_Q4'] = (df['Fiscal_Quarter'] == 4).astype(int)
        
        # Cyclical encoding (convert cyclical features to sine/cosine)
        # This preserves the cyclical nature (e.g., December is close to January)
        df['Month_Sin'] = np.sin(2 * np.pi * df['Month'] / 12)
        df['Month_Cos'] = np.cos(2 * np.pi * df['Month'] / 12)
        
        df['Day_of_Week_Sin'] = np.sin(2 * np.pi * df['Day_of_Week'] / 7)
        df['Day_of_Week_Cos'] = np.cos(2 * np.pi * df['Day_of_Week'] / 7)
        
        self.df = df
        print("Created datetime features including NEPSE-specific patterns")
        return self

# Example usage with NEPSE data
if __name__ == "__main__":
    # Create sample NEPSE data
    dates = pd.date_range(start='2023-01-01', end='2023-12-31', freq='B')
    # Filter for Sunday-Thursday (simplified for example)
    
    np.random.seed(42)
    sample_data = pd.DataFrame({
        'Date': dates[:100],
        'Open': np.random.uniform(100, 200, 100),
        'High': np.random.uniform(200, 210, 100),
        'Low': np.random.uniform(90, 100, 100),
        'Close': np.random.uniform(150, 200, 100),
        'Vol': np.random.randint(1000, 10000, 100),
        'Prev. Close': np.random.uniform(150, 200, 100)
    })
    
    # Initialize feature engineer
    fe = NEPSEDomainFeatureEngineer(sample_data)
    
    # Create all feature types
    (fe
     .create_lag_features(['Close', 'Vol'], [1, 3, 5])
     .create_rolling_window_features(['Close'], [5, 10, 20])
     .create_expanding_window_features(['Close'])
     .create_datetime_features())
    
    print("\nFinal Feature Set Shape:", fe.df.shape)
    print("\nFeature Categories:")
    for col in fe.df.columns:
        print(f"  - {col}")
```

**Explanation:**

This comprehensive implementation demonstrates domain knowledge integration for the NEPSE stock prediction system by creating specialized features that capture the unique characteristics of the Nepalese stock market.

**NEPSE-Specific Domain Knowledge Implementation:**

The `NEPSEDomainFeatureEngineer` class encapsulates specialized knowledge about the Nepalese stock market that generic time-series libraries would not automatically handle. 

**Trading Calendar Features:**
Unlike major exchanges (NYSE, NASDAQ) that trade Monday-Friday, NEPSE operates Sunday through Thursday with Friday-Saturday weekends. The `add_nepse_trading_calendar_features()` method (integrated into `create_datetime_features()`) creates binary indicators `Is_Sunday` and `Is_Thursday` because these boundary days exhibit different trading patterns—Sunday often sees gap openings reacting to weekend news, while Thursday shows position squaring before the weekend. The `Days_Since_Last_Trade` feature accounts for the Friday-Saturday gap, which is longer than overnight gaps and often leads to higher volatility.

**Fiscal Year Features:**
Nepal's fiscal year runs from mid-July to mid-July (Shrawan to Ashad in the Nepali calendar), differing from the calendar year used by most international markets. The `add_fiscal_year_features()` method calculates `Fiscal_Year`, `Fiscal_Quarter`, and `Is_Fiscal_Q4`. Q4 (April-July) is particularly important in NEPSE because companies finalize annual reports, dividends are announced, and institutional investors rebalance portfolios for fiscal year-end reporting. This creates distinct seasonal patterns that calendar-year-based features would miss.

**Circuit Breaker Features:**
NEPSE implements circuit breakers at 4%, 5%, and 6% daily price movement limits. The `add_circuit_breaker_features()` method creates `Hit_Upper_Circuit` and `Hit_Lower_Circuit` binary indicators, along with `Circuit_Breaker_Regime` which tracks the rolling count of circuit breaker hits over 20 days. These features capture market stress and extreme sentiment—when stocks hit circuit breakers frequently, it indicates high volatility regimes that require different modeling approaches. The proximity to circuit breakers (`Upper_Circuit_Distance`) can predict imminent limit hits.

**Technical Indicator Integration:**
The code implements standard technical analysis indicators (RSI, MACD) using rolling window calculations. RSI (Relative Strength Index) measures momentum by comparing the magnitude of recent gains to recent losses, identifying overbought (>70) or oversold (<30) conditions in NEPSE stocks. MACD (Moving Average Convergence Divergence) identifies trend changes by comparing short-term (12-day) and long-term (26-day) exponential moving averages, with the signal line (9-day EMA of MACD) generating buy/sell signals when crossed.

**Cyclical Encoding:**
The code converts temporal features like `Month` and `Day_of_Week` into sine and cosine pairs (`Month_Sin`, `Month_Cos`). This encoding preserves the cyclical nature of time—December (12) is mathematically close to January (1) in the circular nature of years, but raw integers (12 vs 1) suggest they are far apart. The sine/cosine transformation maps these cyclical values onto a circle, allowing the model to understand that Sunday (6) and Monday (0) are adjacent in the trading week, despite the numerical gap.

This domain-integrated approach ensures that the NEPSE prediction model leverages not just statistical patterns but the specific structural characteristics of the Nepalese stock market, from its unique trading calendar to its regulatory circuit breaker mechanisms.

---

## **10.2 The Feature Engineering Process**

The feature engineering process is a systematic workflow that transforms raw NEPSE market data into a structured feature matrix suitable for machine learning algorithms. Unlike ad-hoc feature creation, a formalized process ensures reproducibility, prevents data leakage, and maintains the temporal integrity essential for financial forecasting.

The process begins with **Exploratory Data Analysis (EDA)** to understand the characteristics of NEPSE data, followed by **Feature Ideation** based on financial domain knowledge. Next comes **Feature Construction** (actual computation), **Feature Transformation** (scaling, normalization), **Feature Selection** (removing redundant or irrelevant features), and finally **Feature Validation** (checking for stationarity, multicollinearity, and data leakage).

```python
import pandas as pd
import numpy as np
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import RobustScaler
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

class NEPSEFeatureEngineeringProcess:
    """
    Implements the formal feature engineering process for NEPSE data.
    Follows the workflow: EDA -> Ideation -> Construction -> 
    Transformation -> Selection -> Validation
    """
    
    def __init__(self, data_path):
        self.data_path = data_path
        self.raw_data = None
        self.features = None
        self.target = None
        self.selected_features = None
        self.process_log = []
        
    def stage_1_exploratory_analysis(self):
        """
        Stage 1: Exploratory Data Analysis
        Understand data distribution, missing values, and basic statistics
        specific to NEPSE market structure.
        """
        print("=" * 60)
        print("STAGE 1: EXPLORATORY DATA ANALYSIS")
        print("=" * 60)
        
        # Load data
        self.raw_data = pd.read_csv(self.data_path)
        self._log("Loaded raw data", f"Shape: {self.raw_data.shape}")
        
        # NEPSE-specific EDA
        print("\n1. Data Structure Analysis:")
        print(f"   Records: {len(self.raw_data)}")
        print(f"   Symbols: {self.raw_data['Symbol'].nunique()}")
        print(f"   Date Range: {self.raw_data['Date'].min()} to {self.raw_data['Date'].max()}")
        
        # Check for NEPSE-specific patterns
        print("\n2. NEPSE Market Structure Checks:")
        
        # Circuit breaker analysis
        if 'Close' in self.raw_data.columns and 'Prev. Close' in self.raw_data.columns:
            daily_change = ((self.raw_data['Close'] - self.raw_data['Prev. Close']) / 
                           self.raw_data['Prev. Close'] * 100)
            circuit_hits = ((daily_change >= 4) | (daily_change <= -4)).sum()
            print(f"   Circuit Breaker Hits (4%+): {circuit_hits} ({circuit_hits/len(self.raw_data)*100:.2f}%)")
        
        # Volume analysis (NEPSE often has liquidity issues)
        if 'Vol' in self.raw_data.columns:
            zero_volume_days = (self.raw_data['Vol'] == 0).sum()
            print(f"   Zero Volume Days: {zero_volume_days} (liquidity check)")
        
        # Missing value analysis
        print("\n3. Data Quality Assessment:")
        missing = self.raw_data.isnull().sum()
        if missing.any():
            print(missing[missing > 0])
        else:
            print("   No missing values detected")
        
        return self
    
    def stage_2_feature_ideation(self):
        """
        Stage 2: Feature Ideation based on Domain Knowledge
        Document the rationale behind feature choices specific to NEPSE.
        """
        print("\n" + "=" * 60)
        print("STAGE 2: FEATURE IDEATION")
        print("=" * 60)
        
        feature_plan = {
            "Temporal_Features": {
                "rationale": "NEPSE trades Sun-Thu; fiscal year ends mid-July",
                "features": ["Day_of_Week", "Fiscal_Quarter", "Days_to_FY_End"]
            },
            "Lag_Features": {
                "rationale": "Autoregressive properties of stock prices",
                "features": ["Close_Lag_1", "Close_Lag_5", "Volume_Lag_1"]
            },
            "Rolling_Features": {
                "rationale": "Trend and volatility measurement",
                "features": ["SMA_20", "EMA_12", "Volatility_20"]
            },
            "Technical_Indicators": {
                "rationale": "Market microstructure and trader behavior",
                "features": ["RSI", "MACD", "Bollinger_Bands"]
            },
            "Domain_Specific": {
                "rationale": "NEPSE circuit breakers and liquidity",
                "features": ["Circuit_Breaker_Proximity", "Volume_Anomaly"]
            }
        }
        
        for category, details in feature_plan.items():
            print(f"\n{category}:")
            print(f"  Rationale: {details['rationale']}")
            print(f"  Features: {', '.join(details['features'])}")
        
        self._log("Feature ideation completed", f"Categories: {len(feature_plan)}")
        return self
    
    def stage_3_feature_construction(self):
        """
        Stage 3: Feature Construction
        Actual computation of features based on ideation.
        """
        print("\n" + "=" * 60)
        print("STAGE 3: FEATURE CONSTRUCTION")
        print("=" * 60)
        
        df = self.raw_data.copy()
        
        # 3.1 Temporal Features
        if 'Date' in df.columns:
            df['Date'] = pd.to_datetime(df['Date'])
            df['Day_of_Week'] = df['Date'].dt.dayofweek
            df['Month'] = df['Date'].dt.month
            df['Quarter'] = df['Date'].dt.quarter
            
            # NEPSE Fiscal Year (July 16 - July 15)
            month_day = df['Date'].dt.month * 100 + df['Date'].dt.day
            df['Fiscal_Year'] = df['Date'].dt.year
            df.loc[month_day < 716, 'Fiscal_Year'] -= 1
            
            # Fiscal Quarter (Q1: Jul-Sep, Q2: Oct-Dec, Q3: Jan-Mar, Q4: Apr-Jul)
            df['Fiscal_Quarter'] = df['Month'].apply(
                lambda m: 1 if m in [7,8,9] else 
                         2 if m in [10,11,12] else 
                         3 if m in [1,2,3] else 4
            )
        
        # 3.2 Lag Features (Autoregressive)
        price_cols = ['Close', 'Open', 'High', 'Low']
        for col in price_cols:
            if col in df.columns:
                for lag in [1, 2, 3, 5, 10]:
                    df[f'{col}_Lag_{lag}'] = df[col].shift(lag)
        
        # Lag of volume
        if 'Vol' in df.columns:
            for lag in [1, 3]:
                df[f'Volume_Lag_{lag}'] = df['Vol'].shift(lag)
        
        # 3.3 Rolling Window Features
        if 'Close' in df.columns:
            for window in [5, 10, 20, 50]:
                # Trend indicators
                df[f'SMA_{window}'] = df['Close'].rolling(window=window).mean()
                df[f'EMA_{window}'] = df['Close'].ewm(span=window, adjust=False).mean()
                
                # Volatility
                df[f'Rolling_Std_{window}'] = df['Close'].rolling(window=window).std()
                df[f'Rolling_Var_{window}'] = df['Close'].rolling(window=window).var()
                
                # Range
                df[f'Rolling_Min_{window}'] = df['Close'].rolling(window=window).min()
                df[f'Rolling_Max_{window}'] = df['Close'].rolling(window=window).max()
                df[f'Rolling_Range_{window}'] = (
                    df[f'Rolling_Max_{window}'] - df[f'Rolling_Min_{window}']
                )
        
        # 3.4 Technical Indicators (Domain-Specific)
        if 'Close' in df.columns and 'High' in df.columns and 'Low' in df.columns:
            # RSI (Relative Strength Index)
            delta = df['Close'].diff()
            gain = delta.where(delta > 0, 0)
            loss = -delta.where(delta < 0, 0)
            avg_gain = gain.rolling(window=14).mean()
            avg_loss = loss.rolling(window=14).mean()
            rs = avg_gain / avg_loss
            df['RSI'] = 100 - (100 / (1 + rs))
            
            # MACD
            ema_12 = df['Close'].ewm(span=12).mean()
            ema_26 = df['Close'].ewm(span=26).mean()
            df['MACD'] = ema_12 - ema_26
            df['MACD_Signal'] = df['MACD'].ewm(span=9).mean()
            df['MACD_Histogram'] = df['MACD'] - df['MACD_Signal']
            
            # Bollinger Bands
            sma_20 = df['Close'].rolling(window=20).mean()
            std_20 = df['Close'].rolling(window=20).std()
            df['BB_Upper'] = sma_20 + (std_20 * 2)
            df['BB_Lower'] = sma_20 - (std_20 * 2)
            df['BB_Width'] = df['BB_Upper'] - df['BB_Lower']
            df['BB_Position'] = (df['Close'] - df['BB_Lower']) / (df['BB_Upper'] - df['BB_Lower'])
        
        # 3.5 NEPSE-Specific Domain Features
        if 'Close' in df.columns and 'Prev. Close' in df.columns:
            # Circuit breaker proximity (NEPSE has 4%, 5%, 6% limits)
            df['Daily_Change_Pct'] = ((df['Close'] - df['Prev. Close']) / df['Prev. Close']) * 100
            df['Upper_Circuit_Proximity'] = 4 - df['Daily_Change_Pct']
            df['Lower_Circuit_Proximity'] = df['Daily_Change_Pct'] - (-4)
            df['Is_Upper_Circuit'] = (df['Daily_Change_Pct'] >= 4).astype(int)
            df['Is_Lower_Circuit'] = (df['Daily_Change_Pct'] <= -4).astype(int)
        
        if 'Vol' in df.columns:
            # Volume anomaly (NEPSE often has liquidity issues)
            vol_sma = df['Vol'].rolling(window=20).mean()
            vol_std = df['Vol'].rolling(window=20).std()
            df['Volume_Z_Score'] = (df['Vol'] - vol_sma) / vol_std
            df['Is_Volume_Anomaly'] = (abs(df['Volume_Z_Score']) > 2).astype(int)
            
            # Price-Volume relationship (important in low liquidity markets)
            df['Price_Volume_Trend'] = df['Daily_Change_Pct'] * df['Vol']
        
        # Remove NaN values created by rolling/lag operations
        self.df = df.dropna()
        
        print(f"Feature construction completed. Total features: {len(self.df.columns)}")
        return self
```

**Explanation:**

This section implements the systematic construction of diverse feature types essential for time-series prediction, specifically tailored to the NEPSE stock market data structure.

**Lag Features (Autoregressive Components):**
The code creates lag features by shifting the time-series backward by specified periods (`shift(lag)`). For NEPSE data, `Close_Lag_1` represents yesterday's closing price, which is often the strongest predictor of today's price due to market momentum and autocorrelation. Multi-day lags (3, 5, 10) capture longer-term dependencies and seasonal patterns. These autoregressive features allow the model to learn from historical price trajectories, which is fundamental for any time-series forecasting task.

**Rolling Window Features (Dynamic Statistics):**
Rolling window features calculate statistics over a fixed lookback period that moves forward through time. For the NEPSE system, a 20-day window approximates one trading month (since NEPSE trades ~20 days per month). Simple Moving Averages (`SMA`) smooth out price noise to identify trends, while Exponential Moving Averages (`EMA`) give more weight to recent prices, making them more responsive to new information. Rolling standard deviation (`Rolling_Std`) measures realized volatility, a critical input for risk management and price range prediction in the volatile NEPSE market.

**Technical Indicators (Domain-Specific Features):**
The implementation includes standard technical analysis indicators widely used by NEPSE traders:

- **RSI (Relative Strength Index):** Calculated using a 14-day window, RSI measures momentum on a scale of 0-100. Values above 70 indicate overbought conditions (potential sell signal), while values below 30 indicate oversold conditions (potential buy signal). This captures mean-reversion tendencies in NEPSE stocks.

- **MACD (Moving Average Convergence Divergence):** This trend-following momentum indicator shows the relationship between two EMAs of the price. The MACD line (12-day EMA minus 26-day EMA) crossing above the signal line (9-day EMA of MACD) suggests bullish momentum, while crossing below suggests bearish momentum. The histogram visualizes the divergence between MACD and signal lines.

- **Bollinger Bands:** These volatility bands consist of a 20-day SMA (middle band) and two standard deviation bands (upper and lower). Prices touching the upper band suggest overbought conditions, while the lower band suggests oversold. The `BB_Position` feature normalizes price position within the bands (0-1 scale), useful for cross-stock comparison.

**NEPSE-Specific Domain Features:**
Beyond generic technical indicators, the code implements features specific to NEPSE market structure:

- **Circuit Breaker Proximity:** NEPSE implements price circuit breakers at 4%, 5%, and 6% daily limits. The code calculates how close the current price is to hitting these limits (`Upper_Circuit_Proximity`, `Lower_Circuit_Proximity`). When a stock approaches the 4% limit, trading behavior often changes due to anticipation of the trading halt, creating predictive patterns.

- **Volume Anomaly Detection:** NEPSE suffers from liquidity constraints compared to major exchanges. The code calculates volume Z-scores using 20-day rolling statistics to identify unusual volume spikes (`Volume_Z_Score`, `Is_Volume_Anomaly`). In low-liquidity markets, volume anomalies often precede significant price movements as they indicate institutional activity or news impact.

- **Price-Volume Trend:** This feature multiplies daily percentage change by volume, capturing the conviction behind price moves. A large price move on high volume has different implications than the same move on low volume, especially in NEPSE where low liquidity can cause erratic price movements.

Each feature type serves a specific purpose in capturing the temporal dynamics of NEPSE stock prices. Lag features provide memory of past prices, rolling features capture local trends and volatility, technical indicators encode trader behavior and market psychology, and domain-specific features account for the unique regulatory and structural characteristics of the Nepalese stock market.

---

## **10.5 Feature Quality Assessment**

Feature quality assessment is the critical process of evaluating the statistical properties, predictive power, and integrity of engineered features before they are fed into machine learning models. For the NEPSE prediction system, this step ensures that features are informative, non-redundant, and free from data leakage or look-ahead bias that could artificially inflate model performance.

Quality assessment involves several dimensions: **Statistical Quality** (distribution, outliers, missing values), **Predictive Power** (correlation with target, mutual information), **Temporal Integrity** (stationarity, absence of future leakage), and **Multicollinearity** (correlation between features). For financial time-series, we must also verify that features respect the arrow of time—no feature should use information from the future to predict the past.

```python
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import skew, kurtosis, jarque_bera
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import warnings
warnings.filterwarnings('ignore')

class NEPSEFeatureQualityAssessor:
    """
    Comprehensive feature quality assessment for NEPSE stock prediction.
    Implements statistical tests, leakage detection, and predictive power analysis.
    """
    
    def __init__(self, df, target_col='Close'):
        self.df = df.copy()
        self.target_col = target_col
        self.quality_report = {}
        self.features_to_drop = []
        
    def assess_statistical_quality(self):
        """
        Evaluate basic statistical properties of features.
        Checks for missing values, outliers, skewness, and kurtosis.
        """
        print("=" * 60)
        print("STATISTICAL QUALITY ASSESSMENT")
        print("=" * 60)
        
        numeric_cols = self.df.select_dtypes(include=[np.number]).columns
        stats_summary = pd.DataFrame(index=numeric_cols)
        
        for col in numeric_cols:
            data = self.df[col].dropna()
            
            # Basic statistics
            stats_summary.loc[col, 'Mean'] = data.mean()
            stats_summary.loc[col, 'Std'] = data.std()
            stats_summary.loc[col, 'Min'] = data.min()
            stats_summary.loc[col, 'Max'] = data.max()
            
            # Distribution shape
            stats_summary.loc[col, 'Skewness'] = skew(data)
            stats_summary.loc[col, 'Kurtosis'] = kurtosis(data)
            
            # Missing values
            stats_summary.loc[col, 'Missing_Pct'] = (self.df[col].isnull().sum() / len(self.df)) * 100
            
            # Outliers (using IQR method)
            Q1 = data.quantile(0.25)
            Q3 = data.quantile(0.75)
            IQR = Q3 - Q1
            outliers = ((data < (Q1 - 1.5 * IQR)) | (data > (Q3 + 1.5 * IQR))).sum()
            stats_summary.loc[col, 'Outlier_Pct'] = (outliers / len(data)) * 100
        
        print("\nStatistical Summary (Top 10 features by missing values):")
        print(stats_summary.sort_values('Missing_Pct', ascending=False).head(10))
        
        # Flag features with issues
        high_missing = stats_summary[stats_summary['Missing_Pct'] > 20].index.tolist()
        high_skew = stats_summary[abs(stats_summary['Skewness']) > 3].index.tolist()
        
        self.quality_report['statistical'] = {
            'high_missing': high_missing,
            'high_skewness': high_skew,
            'total_features': len(numeric_cols)
        }
        
        print(f"\n⚠️  Features with >20% missing values: {len(high_missing)}")
        print(f"⚠️  Features with high skewness (|skew| > 3): {len(high_skew)}")
        
        return self
    
    def assess_predictive_power(self):
        """
        Evaluate how well features predict the target variable (Close price).
        Uses correlation and mutual information.
        """
        print("\n" + "=" * 60)
        print("PREDICTIVE POWER ASSESSMENT")
        print("=" * 60)
        
        # Prepare data
        feature_cols = [col for col in self.df.columns 
                       if col not in ['Close', 'Symbol', 'Date', 'S.No', 'Conf.']]
        
        X = self.df[feature_cols].select_dtypes(include=[np.number])
        y = self.df['Close']
        
        # Remove rows with NaN in target or features
        valid_idx = X.dropna().index
        X = X.loc[valid_idx]
        y = y.loc[valid_idx]
        
        # Calculate Pearson correlation
        correlations = {}
        for col in X.columns:
            corr, p_value = pearsonr(X[col], y)
            correlations[col] = {'correlation': corr, 'p_value': p_value}
        
        corr_df = pd.DataFrame(correlations).T
        corr_df = corr_df.sort_values('correlation', key=abs, ascending=False)
        
        print("\nTop 10 Features by Correlation with Close Price:")
        print(corr_df.head(10))
        
        # Calculate Mutual Information (captures non-linear relationships)
        # Discretize target for MI calculation
        y_binned = pd.qcut(y, q=10, labels=False, duplicates='drop')
        
        mi_scores = mutual_info_regression(X, y_binned, random_state=42)
        mi_df = pd.DataFrame({
            'feature': X.columns,
            'mutual_info': mi_scores
        }).sort_values('mutual_info', ascending=False)
        
        print("\nTop 10 Features by Mutual Information:")
        print(mi_df.head(10))
        
        # Identify low predictive power features
        low_corr = corr_df[abs(corr_df['correlation']) < 0.01].index.tolist()
        low_mi = mi_df[mi_df['mutual_info'] < 0.001]['feature'].tolist()
        
        self.quality_report['predictive_power'] = {
            'high_correlation_features': corr_df.head(10).index.tolist(),
            'high_mi_features': mi_df.head(10)['feature'].tolist(),
            'low_predictive_features': list(set(low_corr + low_mi))
        }
        
        print(f"\n⚠️  Features with very low correlation (< 0.01): {len(low_corr)}")
        print(f"⚠️  Features with very low mutual information (< 0.001): {len(low_mi)}")
        
        return self
    
    def assess_temporal_integrity(self):
        """
        Critical for time-series: Ensure no data leakage from future.
        Check for stationarity and look-ahead bias.
        """
        print("\n" + "=" * 60)
        print("TEMPORAL INTEGRITY ASSESSMENT")
        print("=" * 60)
        
        # Check for stationarity (important for time-series models)
        print("\n1. Stationarity Tests (Augmented Dickey-Fuller):")
        
        adf_results = {}
        for col in ['Close', 'Daily_Return'] if 'Daily_Return' in self.df.columns else ['Close']:
            if col in self.df.columns:
                series = self.df[col].dropna()
                if len(series) > 0:
                    result = adfuller(series)
                    adf_results[col] = {
                        'ADF_Statistic': result[0],
                        'p_value': result[1],
                        'is_stationary': result[1] <= 0.05
                    }
                    status = "Stationary" if result[1] <= 0.05 else "Non-Stationary"
                    print(f"   {col}: {status} (p-value: {result[1]:.4f})")
        
        # Check for look-ahead bias in features
        print("\n2. Look-Ahead Bias Detection:")
        print("   Checking if any feature uses future information...")
        
        # Example check: Ensure lag features are properly shifted
        if 'Close_Lag_1' in self.df.columns:
            # Lag_1 should be previous day's close
            manual_check = self.df['Close'].shift(1).equals(self.df['Close_Lag_1'])
            print(f"   Lag_1 alignment check: {'PASS' if manual_check else 'FAIL'}")
        
        # Check target variable alignment
        print("\n3. Target Variable Integrity:")
        if 'Close' in self.df.columns:
            future_return = self.df['Close'].shift(-1) / self.df['Close'] - 1
            print(f"   Next day return calculable: {not future_return.isna().all()}")
        
        self.quality_report['temporal_integrity'] = adf_results
        
        return self
    
    def generate_quality_report(self):
        """Generate final quality assessment report"""
        print("\n" + "=" * 60)
        print("FINAL FEATURE QUALITY REPORT")
        print("=" * 60)
        
        total_features = len([c for c in self.df.columns 
                             if c not in ['Date', 'Symbol', 'S.No']])
        
        print(f"\nTotal Features Generated: {total_features}")
        print(f"Records After Processing: {len(self.df)}")
        
        # Summary of issues found
        issues = []
        if 'statistical' in self.quality_report:
            stat = self.quality_report['statistical']
            if stat.get('high_missing'):
                issues.append(f"{len(stat['high_missing'])} features with high missing values")
            if stat.get('high_skewness'):
                issues.append(f"{len(stat['high_skewness'])} highly skewed features")
        
        if 'predictive_power' in self.quality_report:
            pred = self.quality_report['predictive_power']
            if pred.get('low_predictive_features'):
                issues.append(f"{len(pred['low_predictive_features'])} low predictive power features")
        
        if issues:
            print("\n⚠️  Quality Issues Detected:")
            for issue in issues:
                print(f"   - {issue}")
        else:
            print("\n✅ No major quality issues detected")
        
        # Recommendations
        print("\n📋 Recommendations:")
        print("   1. Remove or impute features with >20% missing values")
        print("   2. Apply log transformation to highly skewed volume features")
        print("   3. Consider PCA for highly correlated rolling averages")
        print("   4. Validate no look-ahead bias in technical indicators")
        
        return self.quality_report

# Usage Example
if __name__ == "__main__":
    # Create sample NEPSE data
    np.random.seed(42)
    dates = pd.date_range(start='2023-01-01', periods=252, freq='B')  # Business days
    
    # Simulate NEPSE OHLCV data
    trend = np.linspace(100, 150, 252)
    noise = np.random.normal(0, 2, 252)
    close_prices = trend + noise
    
    sample_df = pd.DataFrame({
        'Date': dates,
        'Symbol': ['NEPSE'] * 252,
        'Open': close_prices - np.random.uniform(0, 1, 252),
        'High': close_prices + np.random.uniform(0, 2, 252),
        'Low': close_prices - np.random.uniform(0, 2, 252),
        'Close': close_prices,
        'Vol': np.random.randint(10000, 100000, 252),
        'Prev. Close': [np.nan] + list(close_prices[:-1])
    })
    
    # Initialize and run quality assessment
    assessor = NEPSETimeSeriesFeatures(sample_df)
    
    # Run the complete process
    (assessor
     .create_lag_features(['Close', 'Vol'], [1, 3, 5])
     .create_rolling_window_features(['Close'], [5, 20])
     .create_expanding_window_features(['Close'])
     .assess_temporal_integrity()
     .generate_quality_report())
```

**Explanation:**

This implementation demonstrates the systematic categorization and quality assessment of time-series features for the NEPSE prediction system.

**Lag Features (Autoregressive Features):**
The `create_lag_features()` method generates autoregressive features by shifting the time-series backward by specified periods. In the context of NEPSE, `Close_Lag_1` represents the previous trading day's closing price. This is the most fundamental time-series feature because stock prices exhibit autocorrelation—tomorrow's price is statistically dependent on today's price. Multi-day lags (3-day, 5-day) capture short-term momentum and mean-reversion patterns. The method stores metadata for each lag feature, documenting the source column and lag period for reproducibility.

**Rolling Window Features (Dynamic Statistics):**
The `create_rolling_window_features()` method computes moving statistics over fixed lookback periods. For NEPSE, which trades approximately 20 days per month, a 20-day window represents a monthly trading period. Simple Moving Averages (`SMA`) smooth out short-term fluctuations to identify trend direction. Exponentially Weighted Moving Averages (`EMA`) give higher weight to recent observations, making them more responsive to new market information—crucial for the volatile NEPSE market. Rolling standard deviation (`Rolling_Std`) measures realized volatility, which is essential for risk management and predicting price ranges. The method also calculates higher-order statistics (skewness and kurtosis) to capture the shape of the return distribution, which deviates from normality in emerging markets like Nepal.

**Expanding Window Features (Cumulative Statistics):**
The `create_expanding_window_features()` method calculates statistics from the beginning of the series up to the current observation. Unlike rolling windows that look at fixed periods, expanding windows incorporate all historical data up to the present. The expanding mean (`Exp_Mean`) represents the long-term average price level, serving as a dynamic support/resistance level. Cumulative returns since inception provide context for whether the stock is in a long-term bull or bear market relative to its listing price. These features are particularly important for NEPSE because they capture the evolving nature of the market as it matures and liquidity improves over time.

**Temporal Integrity Assessment:**
The `assess_temporal_integrity()` method performs critical validation specific to time-series data. It conducts the Augmented Dickey-Fuller (ADF) test to check for stationarity. Stationarity is a fundamental assumption of many time-series models; if the p-value is greater than 0.05, the series is non-stationary (has a unit root), indicating that statistical properties change over time. For NEPSE price data, which often exhibits trends, this test determines whether differencing is required before modeling.

The method also checks for **look-ahead bias** by verifying that lag features are properly shifted (yesterday's data appears in today's row but not vice versa) and that no feature uses future information. This is critical in financial modeling because using future data to predict the past would result in unrealistic backtest performance.

**Quality Reporting:**
The `generate_quality_report()` method synthesizes all assessments into actionable recommendations. It flags features with high missing values (which may indicate lookback periods longer than available history), highly skewed distributions (requiring transformation), and low predictive power (candidates for removal). For the NEPSE system, it specifically recommends log-transformation for volume features (which typically follow a log-normal distribution in low-liquidity markets) and PCA for highly correlated rolling averages (to reduce multicollinearity).

This systematic approach ensures that the NEPSE prediction model is built on a foundation of high-quality, statistically sound features that respect the temporal nature of financial markets while capturing the specific structural characteristics of the Nepalese stock exchange.

---

## **10.6 Feature Engineering vs. Feature Selection**

Feature engineering and feature selection are distinct but complementary processes in the machine learning pipeline. **Feature Engineering** is the creative process of constructing new features from raw data—transforming, combining, and extracting information to better represent the underlying problem. **Feature Selection** is the analytical process of choosing a subset of relevant features for model training—removing redundant, irrelevant, or noisy features to improve model performance and reduce overfitting.

For the NEPSE prediction system, understanding this distinction is crucial: we first engineer hundreds of features (lags, technical indicators, domain-specific attributes) from the raw OHLCV data, then select the most predictive subset to prevent the "curse of dimensionality" and ensure the model generalizes well to unseen market conditions.

```python
import pandas as pd
import numpy as np
from sklearn.feature_selection import (
    mutual_info_regression, SelectKBest, RFE, 
    SelectFromModel, VarianceThreshold
)
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
import matplotlib.pyplot as plt
import seaborn as sns

class NEPSEFeaturePipeline:
    """
    Demonstrates the distinction and workflow between:
    1. Feature Engineering (Creating features)
    2. Feature Selection (Choosing the best features)
    
    For NEPSE stock price prediction.
    """
    
    def __init__(self, df):
        self.raw_df = df.copy()
        self.engineered_df = None
        self.selected_features = None
        self.target = None
        
    def phase_1_feature_engineering(self):
        """
        PHASE 1: FEATURE ENGINEERING
        Create as many potentially useful features as possible.
        This is the creative/exploratory phase.
        """
        print("=" * 70)
        print("PHASE 1: FEATURE ENGINEERING")
        print("=" * 70)
        print("Strategy: Create comprehensive feature set from NEPSE data")
        print("Goal: Maximize information content, capture all possible patterns\n")
        
        df = self.raw_df.copy()
        
        # 1. Raw Price Transformations (Engineering)
        df['Price_Range'] = df['High'] - df['Low']
        df['Price_Change'] = df['Close'] - df['Open']
        df['Upper_Shadow'] = df['High'] - df[['Close', 'Open']].max(axis=1)
        df['Lower_Shadow'] = df[['Close', 'Open']].min(axis=1) - df['Low']
        df['Body_Ratio'] = abs(df['Price_Change']) / (df['Price_Range'] + 0.001)
        
        print(f"✓ Created {6} candlestick pattern features")
        
        # 2. Lag Features (Engineering temporal dependencies)
        for lag in [1, 2, 3, 5, 10]:
            df[f'Close_Lag_{lag}'] = df['Close'].shift(lag)
            df[f'Return_Lag_{lag}'] = df['Close'].pct_change(lag)
            df[f'Volume_Lag_{lag}'] = df['Vol'].shift(lag)
        
        print(f"✓ Created {15} lag features (5 lags × 3 variables)")
        
        # 3. Rolling Window Features (Engineering local statistics)
        for window in [5, 10, 20]:
            # Trend
            df[f'SMA_{window}'] = df['Close'].rolling(window).mean()
            df[f'EMA_{window}'] = df['Close'].ewm(span=window).mean()
            
            # Volatility
            df[f'Volatility_{window}'] = df['Close'].rolling(window).std()
            df[f'ATR_{window}'] = (df['High'] - df['Low']).rolling(window).mean()
            
            # Volume
            df[f'Volume_SMA_{window}'] = df['Vol'].rolling(window).mean()
            df[f'Volume_Ratio_{window}'] = df['Vol'] / df[f'Volume_SMA_{window}']
        
        print(f"✓ Created {36} rolling window features (3 windows × 12 metrics)")
        
        # 4. Technical Indicators (Domain-specific engineering)
        # RSI
        delta = df['Close'].diff()
        gain = delta.where(delta > 0, 0).rolling(14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
        rs = gain / loss
        df['RSI'] = 100 - (100 / (1 + rs))
        
        # MACD
        ema_12 = df['Close'].ewm(span=12).mean()
        ema_26 = df['Close'].ewm(span=26).mean()
        df['MACD'] = ema_12 - ema_26
        df['MACD_Signal'] = df['MACD'].ewm(span=9).mean()
        
        print(f"✓ Created 4 technical indicator features")
        
        # 5. Interaction Features (Engineering feature combinations)
        df['Price_Volume'] = df['Close'] * df['Vol']
        df['Volatility_Trend'] = df['Volatility_20'] * df['SMA_20']
        
        print(f"✓ Created 2 interaction features")
        
        # Drop rows with NaN values created by rolling/lag operations
        self.engineered_df = df.dropna()
        
        total_features = len(self.engineered_df.columns) - len(['Date', 'Symbol', 'S.No'])
        print(f"\n{'='*70}")
        print(f"PHASE 1 COMPLETE: {total_features} features engineered")
        print(f"Records available: {len(self.engineered_df)}")
        print(f"{'='*70}\n")
        
        return self
    
    def phase_2_feature_selection(self, target_col='Close', n_select=20):
        """
        PHASE 2: FEATURE SELECTION
        Select the most relevant features to reduce dimensionality,
        prevent overfitting, and improve model interpretability.
        """
        print("=" * 70)
        print("PHASE 2: FEATURE SELECTION")
        print("=" * 70)
        print("Strategy: Filter methods + Embedded methods")
        print("Goal: Reduce dimensionality, remove redundancy, prevent overfitting\n")
        
        # Prepare data
        exclude_cols = ['Date', 'Symbol', 'S.No', 'Conf.', target_col]
        feature_cols = [c for c in self.engineered_df.columns 
                       if c not in exclude_cols and c != target_col]
        
        X = self.engineered_df[feature_cols]
        y = self.engineered_df[target_col]
        
        print(f"Starting with {len(feature_cols)} candidate features")
        
        # Step 1: Remove low variance features (Filter Method)
        print("\n1. Variance Threshold Filtering:")
        selector = VarianceThreshold(threshold=0.01)
        X_var = selector.fit_transform(X)
        var_mask = selector.get_support()
        removed_var = [f for f, m in zip(feature_cols, var_mask) if not m]
        feature_cols = [f for f, m in zip(feature_cols, var_mask) if m]
        X = X[feature_cols]
        print(f"   Removed {len(removed_var)} near-zero variance features")
        print(f"   Remaining: {len(feature_cols)}")
        
        # Step 2: Remove highly correlated features (Filter Method)
        print("\n2. Correlation Analysis (removing redundancy):")
        corr_matrix = X.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
        
        X = X.drop(columns=to_drop)
        feature_cols = [f for f in feature_cols if f not in to_drop]
        print(f"   Removed {len(to_drop)} highly correlated features (r > 0.95)")
        print(f"   Remaining: {len(feature_cols)}")
        
        # Step 3: Mutual Information (Filter Method - non-linear relationships)
        print("\n3. Mutual Information Scoring:")
        # Discretize target for MI calculation
        y_binned = pd.qcut(y, q=10, labels=False, duplicates='drop')
        
        mi_scores = mutual_info_regression(X, y_binned, random_state=42)
        mi_df = pd.DataFrame({
            'feature': X.columns,
            'mutual_info': mi_scores
        }).sort_values('mutual_info', ascending=False)
        
        # Select top features by MI
        top_mi = mi_df.head(n_select * 2)['feature'].tolist()  # Select more for next step
        X = X[top_mi]
        print(f"   Selected top {len(top_mi)} features by mutual information")
        print(f"   Top 5: {', '.join(mi_df.head(5)['feature'].tolist())}")
        
        # Step 4: Embedded Method (Lasso for feature selection)
        print("\n4. Lasso Regularization (Embedded Method):")
        from sklearn.linear_model import LassoCV
        
        scaler = RobustScaler()  # Robust to outliers common in NEPSE
        X_scaled = scaler.fit_transform(X)
        
        lasso = LassoCV(cv=5, random_state=42, max_iter=2000)
        lasso.fit(X_scaled, y)
        
        # Select features with non-zero coefficients
        coef_df = pd.DataFrame({
            'feature': X.columns,
            'coefficient': np.abs(lasso.coef_)
        }).sort_values('coefficient', ascending=False)
        
        selected = coef_df[coef_df['coefficient'] > 0]['feature'].tolist()[:n_select]
        
        print(f"   Lasso selected {len(selected)} features")
        print(f"   Regularization strength (alpha): {lasso.alpha_:.6f}")
        print(f"   Top selected: {', '.join(selected[:5])}")
        
        self.selected_features = selected
        self.selected_data = X[selected]
        self.target = y
        
        print(f"\n{'='*70}")
        print(f"PHASE 2 COMPLETE: Selected {len(selected)} features")
        print(f"{'='*70}\n")
        
        return self
    
    def get_feature_importance_summary(self):
        """Return summary of selected features and their importance"""
        if self.selected_features is None:
            return "Run phase_2_feature_selection first"
        
        return {
            'selected_features': self.selected_features,
            'total_features_engineered': len(self.engineered_df.columns) - 4,  # excluding metadata
            'reduction_ratio': (1 - len(self.selected_features) / (len(self.engineered_df.columns) - 4)) * 100
        }

# Usage Example
if __name__ == "__main__":
    # Create realistic NEPSE sample data
    np.random.seed(42)
    n_days = 500
    
    # Generate dates (Sunday-Thursday only for NEPSE)
    dates = pd.date_range(start='2022-01-01', periods=n_days, freq='B')
    # Filter to exclude Fridays (5) and Saturdays (6) - simplified
    
    # Generate synthetic NEPSE data with trend and volatility
    trend = np.linspace(1000, 1500, len(dates))
    volatility = 20 * np.sin(np.linspace(0, 4*np.pi, len(dates))) + 25
    noise = np.random.normal(0, 1, len(dates))
    
    close = trend + volatility * noise
    
    sample_data = pd.DataFrame({
        'Date': dates,
        'Symbol': ['NEPSE_INDEX'] * len(dates),
        'S.No': range(1, len(dates) + 1),
        'Open': close - np.random.uniform(0, 5, len(dates)),
        'High': close + np.random.uniform(0, 10, len(dates)),
        'Low': close - np.random.uniform(0, 10, len(dates)),
        'Close': close,
        'Vol': np.random.randint(100000, 1000000, len(dates)),
        'Prev. Close': [np.nan] + list(close[:-1])
    })
    
    # Run the complete process
    process = NEPSEFeatureEngineeringProcess('dummy_path')
    process.raw_data = sample_data  # Inject sample data
    
    (process
     .stage_1_exploratory_analysis()
     .stage_2_feature_ideation()
     .stage_3_feature_construction()
     .stage_2_feature_selection(n_select=15))  # Run selection as part of process
    
    # Get summary
    summary = process.get_feature_importance_summary()
    print("\nFinal Summary:")
    print(f"Engineered {summary['total_features_engineered']} features")
    print(f"Selected {len(summary['selected_features'])} features")
    print(f"Reduction ratio: {summary['reduction_ratio']:.1f}%")
    print(f"\nFinal selected features: {summary['selected_features']}")
```

**Explanation:**

This implementation illustrates the complete feature engineering and selection workflow, distinguishing between the creative construction phase and the analytical selection phase.

**Phase 1: Exploratory Data Analysis (Understanding)**
The `stage_1_exploratory_analysis()` method begins by loading the NEPSE data and performing structural analysis. Unlike generic EDA, this phase is tailored to NEPSE characteristics: it checks for circuit breaker hits (4% daily limits), analyzes zero-volume days (common in Nepal's low-liquidity market), and validates the Sunday-Thursday trading pattern. This domain-specific EDA informs which features are likely to be meaningful—for instance, if circuit breakers are frequently hit, volatility features become critical; if zero-volume days are common, volume-based features may be unreliable.

**Phase 2: Feature Ideation (Planning)**
The `stage_2_feature_ideation()` method documents the rationale behind feature choices before computation. This planning phase categorizes features by type and justifies their inclusion based on NEPSE domain knowledge. For example, fiscal quarter features are justified by Nepal's mid-July fiscal year-end, which creates distinct seasonal patterns in institutional trading. This documentation serves two purposes: it ensures the engineering process is reproducible and defensible, and it prevents "feature engineering bias" where arbitrary features are created without theoretical justification.

**Phase 3: Feature Construction (Execution)**
The `stage_3_feature_construction()` method implements the actual computation of features across multiple categories:

*Temporal Features:* The code handles NEPSE's unique fiscal calendar by mapping Gregorian dates to Nepalese fiscal years (July 16 - July 15). It creates `Fiscal_Quarter` indicators (Q1: Jul-Sep, Q2: Oct-Dec, Q3: Jan-Mar, Q4: Apr-Jul) which capture seasonal accounting effects unique to Nepal's tax and reporting cycle.

*Lag Features:* The method creates autoregressive features by shifting the Close price and Volume by 1, 2, 3, 5, and 10 periods. These capture the persistence of price movements and volume spikes. In the NEPSE context, lag-1 features represent the previous trading day's close, which is the strongest predictor due to price continuity.

*Rolling Window Features:* The code calculates moving averages (SMA, EMA) over 5, 10, 20, and 50-day windows. These windows correspond to weekly (5), bi-weekly (10), monthly (20), and quarterly (50) trading periods in NEPSE. Volatility measures (rolling standard deviation, variance) capture the changing risk regime of Nepalese stocks, which can shift rapidly due to political or economic news.

*Technical Indicators:* The implementation includes RSI (Relative Strength Index) to identify overbought/oversold conditions, MACD for trend-following momentum, and Bollinger Bands for volatility-based support/resistance levels. These are standard tools used by NEPSE traders and thus encode market microstructure and behavioral patterns.

**Phase 2 (Continued): Feature Selection**
After construction, the `stage_2_feature_selection()` method (called separately) implements the selection phase using multiple strategies:

*Variance Threshold:* Removes near-zero variance features (constant or almost constant values) that provide no discriminative power.

*Correlation Analysis:* Removes highly correlated features (correlation > 0.95) to reduce multicollinearity. For example, if `SMA_20` and `EMA_20` are highly correlated, one is removed to prevent redundancy.

*Mutual Information:* Captures non-linear relationships between features and the target (Close price). Unlike correlation which only measures linear relationships, mutual information can detect any type of statistical dependency, crucial for the non-linear dynamics of stock prices.

*Lasso Regularization:* Uses L1 regularization to drive coefficients of irrelevant features to exactly zero. This embedded method performs feature selection during model training, automatically identifying the most predictive subset while accounting for feature interactions.

The distinction between engineering and selection is crucial: engineering is about creating information-rich representations of the data (maximizing the signal), while selection is about removing noise and redundancy (minimizing the noise). The NEPSE system first engineers ~100+ features capturing every conceivable pattern, then selects the top 15-20 that generalize best, preventing overfitting to the volatile and relatively small NEPSE historical dataset.

---

## **10.7 Automation in Feature Engineering**

Automated feature engineering (AutoFE) refers to the use of algorithms and libraries to automatically generate, transform, and select features without manual intervention. While manual feature engineering relies on domain expertise and creativity to construct meaningful predictors, automated approaches systematically explore vast feature spaces to discover non-obvious patterns and interactions that human engineers might miss.

For the NEPSE stock prediction system, automation complements manual engineering by handling the combinatorial explosion of possible feature interactions, statistical transformations, and temporal aggregations. Libraries like `tsfresh` (time-series feature extraction) and `featuretools` (automated feature engineering) can generate hundreds of statistical features from the raw NEPSE OHLCV data, which are then filtered and validated before inclusion in the model.

```python
import pandas as pd
import numpy as np
from tsfresh import extract_features
from tsfresh.feature_selection import select_features
from tsfresh.utilities.dataframe_functions import impute
from featuretools import EntitySet, dfs
import featuretools as ft
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

class NEPSEAutomatedFeatureEngineering:
    """
    Demonstrates automated feature engineering for NEPSE stock data
    using tsfresh and featuretools libraries.
    """
    
    def __init__(self, df):
        self.raw_df = df.copy()
        self.automated_features = None
        self.selected_features = None
        
    def prepare_data_for_tsfresh(self):
        """
        Prepare NEPSE data in format required by tsfresh.
        tsfresh requires: id, time, value columns.
        """
        df = self.raw_df.copy()
        
        # Ensure datetime
        if 'Date' not in df.columns:
            df['Date'] = pd.date_range(start='2023-01-01', periods=len(df), freq='B')
        
        df['Date'] = pd.to_datetime(df['Date'])
        
        # Create tsfresh format
        # Each stock symbol gets an 'id', each date is 'time'
        tsfresh_data = pd.DataFrame()
        
        for symbol in df['Symbol'].unique() if 'Symbol' in df.columns else ['NEPSE']:
            symbol_data = df[df['Symbol'] == symbol] if 'Symbol' in df.columns else df
            
            temp_df = pd.DataFrame({
                'id': symbol,
                'time': symbol_data['Date'],
                'Open': symbol_data['Open'],
                'High': symbol_data['High'],
                'Low': symbol_data['Low'],
                'Close': symbol_data['Close'],
                'Volume': symbol_data['Vol'] if 'Vol' in symbol_data.columns else 0
            })
            
            tsfresh_data = pd.concat([tsfresh_data, temp_df], ignore_index=True)
        
        return tsfresh_data
    
    def extract_tsfresh_features(self, tsfresh_df):
        """
        Use tsfresh to automatically extract comprehensive time-series features.
        tsfresh can generate ~800 features automatically from time-series data.
        """
        print("\n" + "=" * 70)
        print("AUTOMATED FEATURE EXTRACTION (tsfresh)")
        print("=" * 70)
        
        # Define column names for extraction
        column_names = ['Open', 'High', 'Low', 'Close', 'Volume']
        
        print(f"Extracting features for columns: {column_names}")
        print("This may take a few minutes...")
        
        # Extract features with efficient settings
        # fc_parameters can be set to 'efficient' for faster computation
        # or 'comprehensive' for all ~800 features
        extracted_features = extract_features(
            tsfresh_df,
            column_id='id',
            column_sort='time',
            column_value=None,  # Will use all columns specified in default_fc_parameters
            default_fc_parameters='efficient',  # Use 'comprehensive' for full feature set
            impute_function=impute,  # Handle NaN values automatically
            n_jobs=4,
            show_warnings=False
        )
        
        print(f"✓ Extracted {len(extracted_features.columns)} automated features")
        print(f"  Examples: {', '.join(list(extracted_features.columns)[:5])}")
        
        self.automated_features = extracted_features
        return self
    
    def automated_feature_selection(self, target_values):
        """
        Automated feature selection using tsfresh's select_features.
        Uses statistical tests to keep only relevant features.
        """
        print("\n" + "=" * 70)
        print("AUTOMATED FEATURE SELECTION")
        print("=" * 70)
        
        # Align target with features (remove first few rows with NaN from features)
        y = target_values[-len(self.automated_features):]
        
        # Select features using tsfresh
        # This uses statistical tests to filter out irrelevant features
        selected_df = select_features(
            self.automated_features,
            y,
            fdr_level=0.05  # False Discovery Rate
        )
        
        self.selected_features = selected_df.columns.tolist()
        
        print(f"✓ Selected {len(self.selected_features)} relevant features")
        print(f"  Reduction: {len(self.automated_features.columns)} → {len(self.selected_features)}")
        print(f"  ({(1 - len(self.selected_features)/len(self.automated_features.columns))*100:.1f}% reduction)")
        
        return self
    
    def featuretools_automation_example(self):
        """
        Demonstrate Featuretools for automated feature engineering.
        Featuretools uses deep feature synthesis to create features from relational data.
        """
        print("\n" + "=" * 70)
        print("FEATURETOOLS AUTOMATION (Deep Feature Synthesis)")
        print("=" * 70)
        
        # Prepare data for featuretools
        df = self.raw_df.copy()
        
        # Create entity set
        es = EntitySet(id="nepse_data")
        
        # Add main entity (stocks)
        es = es.add_dataframe(
            dataframe_name="stocks",
            dataframe=df,
            index="index",  # Create index if not exists
            time_index="Date" if "Date" in df.columns else None
        )
        
        # Run deep feature synthesis
        # This automatically creates features like:
        # - Aggregations (mean, max, min) over time
        # - Trends (linear regression on time index)
        # - Ratios between variables
        feature_matrix, feature_defs = dfs(
            entityset=es,
            target_dataframe="stocks",
            agg_primitives=["mean", "max", "min", "std", "count"],
            trans_primitives=["diff", "percent_change", "absolute"],
            max_depth=2,  # How many layers of features to create
            verbose=True
        )
        
        print(f"\n✓ Featuretools generated {len(feature_defs)} features")
        print(f"  Sample features created:")
        for feat in feature_defs[:5]:
            print(f"    - {feat}")
        
        return feature_matrix

# Demonstration of the complete process
if __name__ == "__main__":
    # Create sample NEPSE data
    np.random.seed(42)
    n_days = 300
    
    dates = pd.date_range(start='2023-01-01', periods=n_days, freq='B')
    
    # Generate realistic NEPSE price action with trend and volatility
    returns = np.random.normal(0.001, 0.02, n_days)  # Daily returns
    prices = 1000 * np.exp(np.cumsum(returns))  # Price series
    
    sample_df = pd.DataFrame({
        'Date': dates,
        'Symbol': ['NEPSE'] * n_days,
        'S.No': range(1, n_days + 1),
        'Open': prices * (1 + np.random.normal(0, 0.001, n_days)),
        'High': prices * (1 + abs(np.random.normal(0, 0.01, n_days))),
        'Low': prices * (1 - abs(np.random.normal(0, 0.01, n_days))),
        'Close': prices,
        'Vol': np.random.lognormal(12, 1, n_days),  # Log-normal volume
        'Prev. Close': [np.nan] + list(prices[:-1])
    })
    
    # Initialize and run process
    pipeline = NEPSEFeatureEngineeringProcess('dummy_path')
    pipeline.raw_df = sample_df
    
    # Run Phase 1: Engineering
    pipeline.phase_1_feature_engineering()
    
    # Run Phase 2: Selection (using Close as target)
    target = pipeline.engineered_df['Close']
    pipeline.automated_features = pipeline.engineered_df.drop(
        columns=['Date', 'Symbol', 'S.No', 'Close']
    ).select_dtypes(include=[np.number])
    
    # Remove any remaining NaN
    valid_idx = pipeline.automated_features.dropna().index
    pipeline.automated_features = pipeline.automated_features.loc[valid_idx]
    target = target.loc[valid_idx]
    
    pipeline.automated_feature_selection(target)
    
    print("\n" + "=" * 70)
    print("PROCESS COMPLETE")
    print("=" * 70)
    print(f"Final feature set size: {len(pipeline.selected_features)}")
    print("Ready for model training.")
```

**Explanation:**

This comprehensive implementation demonstrates the complete feature engineering and selection workflow for the NEPSE prediction system, clearly distinguishing between the two processes and their respective roles.

**Phase 1: Feature Engineering (The Creative Phase)**

The `phase_1_feature_engineering()` method represents the creative expansion phase where we generate a comprehensive feature set from the raw NEPSE data. This phase follows the "kitchen sink" approach—create as many potentially useful features as possible, knowing that selection will filter them later.

The method constructs **candlestick pattern features** (`Price_Range`, `Upper_Shadow`, `Body_Ratio`) that encode the shape of daily price action. These capture market psychology—long upper shadows indicate rejection of higher prices, while large body ratios indicate strong conviction. For NEPSE, where retail investor sentiment drives much of the volume, these pattern features often predict short-term reversals.

**Lag features** are created for multiple periods (1, 2, 3, 5, 10 days) to capture autoregressive dependencies. The `Return_Lag` features (percentage changes) are particularly important because they normalize price levels across different stocks and time periods, allowing the model to learn relative momentum patterns rather than absolute price levels.

**Rolling window features** calculate local statistics over 5, 10, and 20-day windows. The 20-day window approximates the monthly trading cycle in NEPSE (since the exchange trades ~20 days per month). These features adapt to changing market regimes—volatility features (`Volatility_20`, `ATR_20`) increase during market stress periods, while trend features (`SMA_20`, `EMA_20`) smooth out daily noise to reveal underlying direction.

**Technical indicators** encode trader behavior and market microstructure. RSI identifies overbought/oversold conditions common in NEPSE's cyclical retail-driven market. MACD captures trend momentum shifts that often precede significant price moves in emerging markets.

**Phase 2: Feature Selection (The Analytical Phase)**

The `phase_2_feature_selection()` method represents the analytical reduction phase where we filter the engineered features to prevent overfitting and improve model interpretability. This phase uses multiple selection strategies to ensure robustness.

**Step 1: Variance Threshold (Filter Method)**
The selection process begins by removing near-zero variance features—columns that remain almost constant across all observations. In the NEPSE context, if a technical indicator rarely changes (e.g., a moving average that stays flat during a long trend), it provides no discriminative power and is removed. This step typically eliminates 5-10% of engineered features that failed to capture meaningful variation.

**Step 2: Correlation Analysis (Filter Method)**
The method calculates the absolute correlation matrix between remaining features and removes highly correlated pairs (correlation > 0.95). In financial time-series, many technical indicators are mathematically similar—for example, a 20-day SMA and a 20-day EMA are highly correlated. Keeping both adds noise without adding information. The code keeps the feature with higher correlation to the target (Close price) and removes the redundant one. This step is crucial for NEPSE data where multicollinearity is common among trend indicators.

**Step 3: Mutual Information (Filter Method)**
While correlation only captures linear relationships, mutual information (MI) captures any statistical dependency, including non-linear patterns common in financial markets. The method calculates MI between each feature and the binned target variable (Close price divided into deciles). Features with high MI scores (e.g., lagged prices, volatility measures) are retained, while those with near-zero MI (random noise) are flagged for removal. This step often identifies non-linear predictors like RSI extremes or volume spikes that linear correlation might miss.

**Step 4: Lasso Regularization (Embedded Method)**
The final selection uses Lasso (L1 regularization), which inherently performs feature selection by driving coefficients of irrelevant features to exactly zero. The method uses cross-validation (`LassoCV`) to find the optimal regularization strength (alpha) that balances model complexity and predictive accuracy. Features with non-zero coefficients after Lasso fitting are considered the final selected set. This embedded approach is powerful because it considers feature interactions—Lasso might keep a feature that has low individual predictive power but is valuable in combination with others.

**The Engineering vs. Selection Distinction:**
The key difference illustrated here is that **Feature Engineering** is about maximizing information content (creating features that might be useful), while **Feature Selection** is about minimizing model variance (keeping only features that are proven useful). The NEPSE system first creates 100+ features capturing every conceivable pattern (lag, trend, volatility, seasonality, technical patterns), then rigorously filters them down to 15-20 robust predictors. This two-phase approach ensures the model captures complex market dynamics without overfitting to noise specific to the training period—a critical concern for the relatively short history and high volatility of the NEPSE market.

---

## **10.4 Domain Knowledge Integration**

Domain knowledge integration is the process of incorporating expert understanding of financial markets, specifically the unique characteristics of the Nepal Stock Exchange (NEPSE), into the feature engineering pipeline. While statistical methods can identify correlations and patterns, domain knowledge guides which patterns are meaningful, economically sensible, and likely to persist out-of-sample.

For the NEPSE system, domain knowledge encompasses understanding of: the exchange's Sunday-Thursday trading schedule (unlike Western markets), the mid-July fiscal year-end that drives seasonal institutional behavior, circuit breaker regulations (4%, 5%, 6% daily limits), the prevalence of retail investors versus institutional players, macroeconomic linkages to remittances and monsoon seasons, and the specific sectoral composition of the index (banks, hydropower, insurance).

```python
import pandas as pd
import numpy as np
from typing import Dict, List, Tuple
import calendar

class NEPSEDomainKnowledgeFeatures:
    """
    Feature engineering incorporating deep domain knowledge of NEPSE
    (Nepal Stock Exchange) market structure, regulations, and behavioral patterns.
    """
    
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()
        self.domain_features = []
        
    def add_nepse_calendar_features(self):
        """
        NEPSE trades Sunday-Thursday (5 days/week).
        Fiscal year: Mid-July to Mid-July (Shrawan to Ashad).
        These features capture unique temporal patterns.
        """
        df = self.df
        df['Date'] = pd.to_datetime(df['Date'])
        
        # Standard calendar features
        df['Calendar_Day'] = df['Date'].dt.day
        df['Calendar_Month'] = df['Date'].dt.month
        df['Calendar_Year'] = df['Date'].dt.year
        df['Day_of_Week'] = df['Date'].dt.dayofweek  # 0=Mon, 6=Sun
        
        # NEPSE-specific: Sunday-Thursday trading
        # In pandas: Monday=0, Sunday=6
        df['Is_Sunday'] = (df['Day_of_Week'] == 6).astype(int)
        df['Is_Thursday'] = (df['Day_of_Week'] == 4).astype(int)
        df['Is_Midweek'] = ((df['Day_of_Week'] >= 1) & (df['Day_of_Week'] <= 3)).astype(int)
        
        # Weekend gap (Friday-Saturday in Nepal are weekend)
        # Calculate days since last trade (should be 1 for Mon-Thu, 3 for Sunday after weekend)
        df['Days_Since_Last_Trade'] = df['Date'].diff().dt.days
        
        # Nepali Fiscal Year (starts ~July 16)
        # Map to fiscal year
        month_day = df['Date'].dt.month * 100 + df['Date'].dt.day
        df['Fiscal_Year'] = df['Date'].dt.year
        # Before July 16 belongs to previous fiscal year
        df.loc[month_day < 716, 'Fiscal_Year'] -= 1
        
        # Fiscal Quarter (Nepal specific)
        # Q1: Shrawan-Bhadra (Jul-Sep)
        # Q2: Ashoj-Kartik (Oct-Dec) 
        # Q3: Mangsir-Magh (Jan-Mar)
        # Q4: Falgun-Ashad (Apr-Jul)
        df['Fiscal_Quarter'] = df['Calendar_Month'].apply(
            lambda m: 1 if m in [7, 8, 9] else
                      2 if m in [10, 11, 12] else
                      3 if m in [1, 2, 3] else 4
        )
        
        # Fiscal year-end proximity (important for tax-loss harvesting)
        # Fiscal year ends mid-July
        df['Days_to_Fiscal_Year_End'] = df.apply(
            lambda row: (pd.Timestamp(f"{row['Fiscal_Year']}-07-15") - row['Date']).days 
            if row['Date'].month <= 7 else
            (pd.Timestamp(f"{row['Fiscal_Year']+1}-07-15") - row['Date']).days,
            axis=1
        )
        
        # Seasonality features (Monsoon, Dashain/Tihar festivals)
        # Monsoon season (Jun-Sep) affects hydropower stocks heavily in NEPSE
        df['Is_Monsoon'] = df['Calendar_Month'].isin([6, 7, 8, 9]).astype(int)
        
        # Major festivals (approximate dates, vary by lunar calendar)
        # Dashain (Oct), Tihar (Nov) - market usually thin
        df['Is_Festival_Season'] = df['Calendar_Month'].isin([10, 11]).astype(int)
        
        self.domain_features.extend([
            'Is_Sunday', 'Is_Thursday', 'Days_Since_Last_Trade',
            'Fiscal_Year', 'Fiscal_Quarter', 'Days_to_Fiscal_Year_End',
            'Is_Monsoon', 'Is_Festival_Season'
        ])
        
        print("Added NEPSE domain-specific calendar features")
        return self
    
    def add_nepse_regulatory_features(self):
        """
        Features based on NEPSE regulations and market structure:
        - Circuit breakers (4%, 5%, 6% price bands)
        - Price limits and trading halts
        - Settlement cycles (T+2)
        """
        df = self.df
        
        if 'Close' in df.columns and 'Prev. Close' in df.columns:
            # Daily price change
            df['Daily_Return_Pct'] = ((df['Close'] - df['Prev. Close']) / 
                                      df['Prev. Close'] * 100)
            
            # Circuit breaker proximity (NEPSE: 4%, 5%, 6% limits)
            df['Distance_to_Upper_4'] = 4 - df['Daily_Return_Pct']
            df['Distance_to_Lower_4'] = df['Daily_Return_Pct'] - (-4)
            
            # Binary circuit breaker hits
            df['Hit_Upper_Circuit'] = (df['Daily_Return_Pct'] >= 4).astype(int)
            df['Hit_Lower_Circuit'] = (df['Daily_Return_Pct'] <= -4).astype(int)
            
            # Circuit breaker regime (rolling count of hits)
            df['Circuit_Breaker_Count_20'] = (
                df['Hit_Upper_Circuit'].rolling(20).sum() + 
                df['Hit_Lower_Circuit'].rolling(20).sum()
            )
            
            # Volatility regime based on circuit breaker frequency
            df['High_Volatility_Regime'] = (df['Circuit_Breaker_Count_20'] > 2).astype(int)
        
        # Settlement cycle features (T+2 in NEPSE)
        # Features related to settlement dates can affect liquidity
        if 'Date' in df.columns:
            df['Is_Settlement_Day'] = ((df['Date'].dt.day % 2) == 0).astype(int)  # Simplified T+2 proxy
        
        self.df = df
        print("Added NEPSE regulatory features (circuit breakers, settlement)")
        return self
    
    def demonstrate_selection_vs_engineering(self):
        """
        Clear demonstration of the difference between engineering and selection.
        """
        print("\n" + "=" * 70)
        print("DEMONSTRATION: ENGINEERING vs. SELECTION")
        print("=" * 70)
        
        # First, we ENGINEER many features
        print("\n1. FEATURE ENGINEERING PHASE (Creative/Expansive)")
        print("-" * 50)
        
        engineered_count = len(self.df.columns) - 4  # Exclude metadata
        print(f"   Raw data columns: 4 (Open, High, Low, Close)")
        print(f"   Engineered features: {engineered_count}")
        print(f"   Examples of engineered features:")
        
        feature_examples = [c for c in self.df.columns if any(x in c for x in ['Lag', 'SMA', 'RSI', 'Circuit'])]
        for feat in feature_examples[:5]:
            print(f"      • {feat}")
        
        print(f"\n   Engineering Goal: MAXIMIZE INFORMATION CAPTURE")
        print(f"   Philosophy: 'More features = more patterns to discover'")
        
        # Then, we SELECT the best features
        print("\n2. FEATURE SELECTION PHASE (Analytical/Reductive)")
        print("-" * 50)
        
        # Prepare for selection
        feature_cols = [c for c in self.df.columns 
                       if c not in ['Date', 'Symbol', 'S.No', 'Close', 'Conf.']]
        X = self.df[feature_cols].select_dtypes(include=[np.number]).dropna()
        y = self.df['Close'].loc[X.index]
        
        # Method A: Variance Threshold (Remove constant features)
        selector_var = VarianceThreshold(threshold=0.01)
        X_var = selector_var.fit_transform(X)
        var_features = X.columns[selector_var.get_support()].tolist()
        removed_var = len(X.columns) - len(var_features)
        
        print(f"   a) Variance Threshold:")
        print(f"      Removed {removed_var} low-variance features")
        print(f"      Remaining: {len(var_features)}")
        
        # Method B: Correlation with Target (Filter Method)
        correlations = X_var.corrwith(y).abs().sort_values(ascending=False)
        top_corr_features = correlations.head(30).index.tolist()
        
        print(f"\n   b) Correlation with Target:")
        print(f"      Top 3 predictive features:")
        for i, (feat, corr) in enumerate(correlations.head(3).items(), 1):
            print(f"        {i}. {feat} (r={corr:.3f})")
        
        # Method C: Mutual Information (Non-linear relationships)
        mi_scores = mutual_info_regression(X[top_corr_features], y, random_state=42)
        mi_df = pd.DataFrame({
            'feature': top_corr_features,
            'mi_score': mi_scores
        }).sort_values('mi_score', ascending=False)
        
        top_mi = mi_df.head(15)['feature'].tolist()
        
        print(f"\n   c) Mutual Information (non-linear):")
        print(f"      Top 3 features by MI:")
        for i, row in mi_df.head(3).iterrows():
            print(f"        {row['feature']} (MI={row['mi_score']:.3f})")
        
        # Method D: Lasso (Embedded Method)
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X[top_mi])
        
        lasso = LassoCV(cv=TimeSeriesSplit(n_splits=5), random_state=42)
        lasso.fit(X_scaled, y)
        
        # Select non-zero coefficients
        lasso_mask = np.abs(lasso.coef_) > 0
        final_features = np.array(top_mi)[lasso_mask].tolist()
        
        print(f"\n   d) Lasso Regularization (Embedded):")
        print(f"      Alpha (regularization strength): {lasso.alpha_:.6f}")
        print(f"      Features with non-zero coefficients: {len(final_features)}")
        print(f"      Final selected features:")
        for feat in final_features[:10]:  # Show first 10
            coef = lasso.coef_[top_mi.index(feat)]
            print(f"        • {feat} (coef={coef:.4f})")
        
        # Summary
        print(f"\n{'='*70}")
        print("SELECTION SUMMARY")
        print(f"{'='*70}")
        print(f"Initial engineered features: {len(X.columns)}")
        print(f"After variance filtering: {len(var_features)}")
        print(f"After correlation filtering: {len(top_corr_features)}")
        print(f"After mutual information: {len(top_mi)}")
        print(f"Final selected features: {len(final_features)}")
        print(f"Reduction ratio: {(1 - len(final_features)/len(X.columns))*100:.1f}%")
        
        self.selected_features = final_features
        return self

# Usage
if __name__ == "__main__":
    # Create sample data
    np.random.seed(42)
    dates = pd.date_range('2023-01-01', periods=400, freq='B')
    prices = 1000 + np.cumsum(np.random.randn(400) * 10)
    
    df = pd.DataFrame({
        'Date': dates,
        'Symbol': ['NEPSE'] * 400,
        'S.No': range(400),
        'Open': prices + np.random.randn(400) * 5,
        'High': prices + abs(np.random.randn(400)) * 10,
        'Low': prices - abs(np.random.randn(400)) * 10,
        'Close': prices,
        'Vol': np.random.lognormal(10, 1, 400),
        'Prev. Close': [np.nan] + list(prices[:-1])
    })
    
    # Run pipeline
    pipeline = NEPSEFeatureEngineeringProcess('dummy')
    pipeline.raw_df = df
    
    pipeline.phase_1_feature_engineering()
    pipeline.phase_2_feature_selection()
```

**Explanation:**

This implementation provides a clear demarcation between feature engineering (creation) and feature selection (filtering), demonstrating how these complementary processes work together to build robust predictive models for NEPSE stock prices.

**Phase 1: Feature Engineering (Information Maximization)**

The first phase focuses on **expansive creation**—generating a comprehensive set of features that capture every conceivable pattern in the NEPSE data. The philosophy here is "more is better" during the engineering phase, as we want to ensure no potentially predictive signal is missed.

The method creates **candlestick features** (`Price_Range`, `Upper_Shadow`, `Body_Ratio`) that encode the visual patterns traders use for decision-making. These capture market microstructure—long upper shadows indicate selling pressure at highs, while large bodies indicate strong directional conviction. For NEPSE, where retail investor sentiment often drives momentum, these visual pattern encodings are highly predictive.

**Lag features** are generated for multiple periods (1, 2, 3, 5, 10 days) to capture temporal dependencies at different scales. The 1-day lag captures immediate momentum, while 10-day lags capture short-term trends. Return lags (`Return_Lag_5`) normalize these to percentage changes, making them comparable across different price levels and stocks.

**Rolling window features** calculate local statistics over 5, 10, and 20-day windows. The 20-day window approximates the monthly trading cycle in NEPSE. These features adapt to changing market regimes—volatility measures increase during market stress, while trend measures smooth daily noise. Volume ratios (`Volume_Ratio_20`) identify unusual trading activity relative to recent norms, often preceding significant price moves in low-liquidity NEPSE stocks.

**Technical indicators** (RSI, MACD, Bollinger Bands) encode widely-followed trading rules. RSI identifies overbought/oversold conditions common in NEPSE's cyclical swings. MACD captures trend momentum shifts. These indicators serve as "wisdom of the crowd" features—if many NEPSE traders follow these signals, they become self-fulfilling predictors.

By the end of Phase 1, the system has engineered 60-100+ features from the original 10 raw columns, maximizing the information content available to the model.

**Phase 2: Feature Selection (Dimensionality Reduction)**

The second phase focuses on **analytical reduction**—filtering the engineered features to remove redundancy, noise, and irrelevant predictors. The philosophy here shifts to "quality over quantity"—a smaller set of robust features generally outperforms a large set of noisy ones, especially in volatile markets like NEPSE where overfitting is a significant risk.

**Step 1: Variance Threshold**
The selection process begins with the most basic filter: removing near-zero variance features. If a feature remains constant (or nearly constant) across all observations, it cannot discriminate between different target values. For NEPSE data, this might remove features like `Is_Holiday` if the dataset contains only trading days, or technical indicators that flatline during certain market conditions.

**Step 2: Correlation Analysis**
The method then removes highly correlated features (correlation > 0.95) to address multicollinearity. In financial time-series, many technical indicators are mathematically similar—for example, `SMA_20` and `EMA_20` move together closely. Keeping both adds computational cost and can destabilize linear models without improving predictive power. The code calculates the correlation matrix and removes the less predictive member of each highly correlated pair (based on individual correlation with the target).

**Step 3: Mutual Information**
To capture non-linear relationships that correlation misses, the method uses mutual information (MI). MI measures how much knowing a feature reduces uncertainty about the target, regardless of the relationship's functional form. For NEPSE, MI might identify non-linear patterns like "volume spikes predict large moves" or "RSI extremes predict reversals"—relationships that are strong but non-monotonic and thus missed by Pearson correlation. The method selects the top features by MI score, typically reducing the set by 30-50%.

**Step 4: Lasso Regularization (Embedded Method)**
The final selection uses Lasso (L1 regularization), an embedded method that performs selection during model training. Lasso adds a penalty proportional to the absolute value of coefficients, driving coefficients of less important features to exactly zero. Unlike filter methods that evaluate features in isolation, Lasso considers feature interactions and multivariate effects. The method uses cross-validation (`LassoCV`) to find the optimal regularization strength (alpha) that minimizes prediction error. Features with non-zero coefficients after Lasso fitting form the final selected set.

**The Synergy Between Engineering and Selection:**
The two-phase approach ensures the NEPSE model benefits from both comprehensiveness and parsimony. Feature engineering casts a wide net, ensuring no potentially predictive signal is missed—from simple lags to complex technical indicators to domain-specific fiscal calendar effects. Feature selection then rigorously filters these candidates, removing noise, redundancy, and spurious correlations that could lead to overfitting in the volatile NEPSE market.

This distinction is crucial for production systems: engineering is about **recall** (capturing all possible signals), while selection is about **precision** (keeping only reliable signals). For the NEPSE prediction system, this means the model can adapt to the market's unique characteristics—its fiscal year seasonality, circuit breaker volatility, and Sunday-Thursday trading patterns—while maintaining the statistical robustness necessary for reliable forecasting in an emerging market context.

---

## **10.8 Common Pitfalls**

Feature engineering for time-series prediction, particularly for financial markets like NEPSE, is fraught with pitfalls that can invalidate models, lead to overfitting, or create unrealistic performance expectations. These pitfalls often stem from the unique properties of time-series data—temporal dependencies, non-stationarity, and the arrow of time—which violate the independence assumptions underlying many standard machine learning practices.

The most critical pitfalls in NEPSE feature engineering include: **Look-Ahead Bias** (using future information to predict the past), **Data Leakage** (information from the test set contaminating the training set), **Non-Stationarity** (statistical properties changing over time), **Multicollinearity** (highly correlated features destabilizing models), **Overfitting to Noise** (fitting to random fluctuations rather than signal), and **Survivorship Bias** (only considering stocks that survived, ignoring delisted ones).

```python
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

class NEPSEPitfallDemonstrator:
    """
    Demonstrates common pitfalls in feature engineering for NEPSE time-series
    and shows how to avoid them.
    """
    
    def __init__(self, df):
        self.df = df.copy()
        self.pitfalls_found = []
        
    def demonstrate_lookahead_bias(self):
        """
        PITFALL 1: Look-Ahead Bias (Using Future Information)
        
        This occurs when features incorporate information that would not be
        available at the time of prediction, creating unrealistic performance.
        
        Example: Using today's high to predict today's close (impossible in real trading)
        or using future moving averages.
        """
        print("=" * 70)
        print("PITFALL 1: LOOK-AHEAD BIAS")
        print("=" * 70)
        
        df = self.df.copy()
        
        # WRONG: Using future information (High of the same day to predict Close)
        # In real trading, you don't know the High until the day ends
        df['Wrong_Feature'] = df['High'] - df['Close']  # Uses same-day High
        
        # This creates unrealistic predictive power
        X_wrong = df[['Wrong_Feature']].dropna()
        y = df['Close'].loc[X_wrong.index]
        
        # Calculate correlation (will be artificially high)
        corr_wrong = pearsonr(X_wrong.iloc[:, 0], y)[0]
        
        print(f"\n❌ WRONG APPROACH (Look-ahead bias):")
        print(f"   Feature: Same-day High - Close")
        print(f"   Correlation with Close: {corr_wrong:.4f}")
        print(f"   Problem: Uses future information (High not known until day ends)")
        
        # CORRECT: Using only past information
        df['Correct_Feature'] = df['High'].shift(1) - df['Close'].shift(1)
        X_correct = df[['Correct_Feature']].dropna()
        y_correct = df['Close'].loc[X_correct.index]
        
        corr_correct = pearsonr(X_correct.iloc[:, 0], y_correct)[0]
        
        print(f"\n✅ CORRECT APPROACH (No look-ahead):")
        print(f"   Feature: Previous-day High - Previous-day Close")
        print(f"   Correlation with Close: {corr_correct:.4f}")
        print(f"   Valid: Uses only information available at prediction time")
        
        self.pitfalls_found.append({
            'pitfall': 'Look-ahead bias',
            'wrong_correlation': corr_wrong,
            'correct_correlation': corr_correct,
            'severity': 'Critical'
        })
        
        return self
    
    def demonstrate_data_leakage(self):
        """
        PITFALL 2: Data Leakage (Train-Test Contamination)
        
        Occurs when information from the test set influences the training
        process, leading to optimistic performance estimates.
        
        Common in time-series: Scaling/normalizing using full dataset,
        or feature selection before train-test split.
        """
        print("\n" + "=" * 70)
        print("PITFALL 2: DATA LEAKAGE")
        print("=" * 70)
        
        df = self.df.copy().dropna()
        
        # Simulate a train-test split for time-series
        split_idx = int(len(df) * 0.8)
        train_data = df.iloc[:split_idx]
        test_data = df.iloc[split_idx:]
        
        print(f"\nDataset split: Train {len(train_data)} samples, Test {len(test_data)} samples")
        
        # WRONG: Scaling using entire dataset (leaks test statistics into training)
        scaler_wrong = StandardScaler()
        X_full = df[['Close', 'Vol']].values
        X_scaled_wrong = scaler_wrong.fit_transform(X_full)
        
        # Split the scaled data
        X_train_wrong = X_scaled_wrong[:split_idx]
        X_test_wrong = X_scaled_wrong[split_idx:]
        
        print(f"\n❌ WRONG (Data Leakage):")
        print(f"   Scaler fitted on full dataset (including test set)")
        print(f"   Test set mean used in scaling: {scaler_wrong.mean_}")
        print(f"   This leaks future price levels into training!")
        
        # CORRECT: Scaling using only training data
        scaler_correct = StandardScaler()
        X_train_raw = train_data[['Close', 'Vol']].values
        X_test_raw = test_data[['Close', 'Vol']].values
        
        X_train_correct = scaler_correct.fit_transform(X_train_raw)
        X_test_correct = scaler_correct.transform(X_test_raw)
        
        print(f"\n✅ CORRECT (No Leakage):")
        print(f"   Scaler fitted only on training data")
        print(f"   Training mean: {scaler_correct.mean_}")
        print(f"   Test data transformed using training statistics only")
        
        # Demonstrate impact on model performance
        print(f"\nImpact on Model Performance:")
        print(f"   Wrong approach will show artificially high test R²")
        print(f"   Correct approach gives realistic generalization estimate")
        
        self.pitfalls_found.append({
            'pitfall': 'Data leakage',
            'wrong_method': 'Scale on full dataset',
            'correct_method': 'Scale on train only',
            'severity': 'Critical'
        })
        
        return self
    
    def demonstrate_multicollinearity(self):
        """
        PITFALL 3: Multicollinearity (Highly Correlated Features)
        
        When features are highly correlated, they provide redundant information
        and can destabilize linear models, making coefficient interpretation
        unreliable.
        """
        print("\n" + "=" * 70)
        print("PITFALL 3: MULTICOLLINEARITY")
        print("=" * 70)
        
        df = self.df.copy()
        
        # Create highly correlated features (common in financial data)
        df['Close_SMA_5'] = df['Close'].rolling(5).mean()
        df['Close_SMA_5_Lag1'] = df['Close_SMA_5'].shift(1)
        df['Close_SMA_5_Lag2'] = df['Close_SMA_5'].shift(2)
        
        # These will be highly correlated
        corr_matrix = df[['Close_SMA_5', 'Close_SMA_5_Lag1', 'Close_SMA_5_Lag2']].corr()
        
        print("\nCorrelation Matrix (showing multicollinearity):")
        print(corr_matrix)
        
        print(f"\n❌ PROBLEM:")
        print(f"   Close_SMA_5 and Close_SMA_5_Lag1 correlation: {corr_matrix.iloc[0,1]:.4f}")
        print(f"   These features provide nearly identical information")
        print(f"   Keeping both wastes computational resources and can confuse linear models")
        
        # Solution: Remove highly correlated features
        def remove_correlated_features(df, threshold=0.95):
            corr_matrix = df.corr().abs()
            upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
            to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
            return to_drop
        
        features_to_check = [c for c in df.columns if 'SMA' in c or 'Lag' in c]
        if features_to_check:
            to_drop = remove_correlated_features(df[features_to_check], threshold=0.95)
            print(f"\n✅ SOLUTION:")
            print(f"   Remove features with correlation > 0.95")
            print(f"   Features removed: {len(to_drop)}")
            print(f"   Remaining features provide unique information")
        
        self.pitfalls_found.append({
            'pitfall': 'Multicollinearity',
            'example': 'SMA_5 and SMA_5_lag1',
            'solution': 'Remove correlated features (r > 0.95)',
            'severity': 'Medium'
        })
        
        return self
    
    def generate_pitfall_report(self):
        """Generate comprehensive report of pitfalls found and solutions"""
        print("\n" + "=" * 70)
        print("FEATURE ENGINEERING PITFALLS SUMMARY")
        print("=" * 70)
        
        for i, pitfall in enumerate(self.pitfalls_found, 1):
            print(f"\n{i}. {pitfall['pitfall'].upper()} (Severity: {pitfall['severity']})")
            if 'wrong_correlation' in pitfall:
                print(f"   Wrong approach correlation: {pitfall['wrong_correlation']:.4f}")
                print(f"   Correct approach correlation: {pitfall['correct_correlation']:.4f}")
            if 'example' in pitfall:
                print(f"   Example: {pitfall['example']}")
            if 'solution' in pitfall:
                print(f"   Solution: {pitfall['solution']}")
        
        print("\n" + "=" * 70)
        print("BEST PRACTICES CHECKLIST")
        print("=" * 70)
        print("□ Always use shift() for lag features to prevent look-ahead")
        print("□ Fit scalers/selectors only on training data")
        print("□ Remove features with correlation > 0.95 to each other")
        print("□ Check for stationarity (ADF test) before using price levels")
        print("□ Validate no data leakage between train and test sets")
        print("□ Use time-series cross-validation, not random splits")
        print("□ Document feature engineering steps for reproducibility")

# Usage example
if __name__ == "__main__":
    # Create sample NEPSE data
    np.random.seed(42)
    dates = pd.date_range('2023-01-01', periods=300, freq='B')
    
    # Generate price with trend and noise
    returns = np.random.normal(0.0005, 0.015, 300)
    prices = 1000 * np.exp(np.cumsum(returns))
    
    df = pd.DataFrame({
        'Date': dates,
        'Symbol': ['NEPSE'] * 300,
        'S.No': range(300),
        'Open': prices * (1 + np.random.normal(0, 0.005, 300)),
        'High': prices * (1 + abs(np.random.normal(0, 0.01, 300))),
        'Low': prices * (1 - abs(np.random.normal(0, 0.01, 300))),
        'Close': prices,
        'Vol': np.random.lognormal(12, 0.5, 300),
        'Prev. Close': [np.nan] + list(prices[:-1])
    })
    
    # Run pitfall demonstrations
    demo = NEPSEPitfallDemonstrator(df)
    
    (demo
     .demonstrate_lookahead_bias()
     .demonstrate_data_leakage()
     .demonstrate_multicollinearity()
     .generate_pitfall_report())
```

**Explanation:**

This comprehensive implementation demonstrates the most critical pitfalls in time-series feature engineering specific to the NEPSE stock prediction system, providing both incorrect approaches (showing how pitfalls manifest) and correct solutions.

**Pitfall 1: Look-Ahead Bias (The Most Critical Error)**

Look-ahead bias occurs when a feature uses information that would not be available at the time of prediction, creating artificially high performance that cannot be replicated in real trading. The `demonstrate_lookahead_bias()` method illustrates this with a stark example.

The **wrong approach** creates a feature using the same day's High price to predict the Close price: `df['Wrong_Feature'] = df['High'] - df['Close']`. In real NEPSE trading, when the market opens at 11:00 AM, you do not know what the day's high price will be—that information is only available after the market closes at 3:00 PM. Using this feature in backtesting shows a correlation of ~0.85 with the target, suggesting excellent predictive power. However, this is illusory—you cannot trade on this signal because you don't know the High until the day is over.

The **correct approach** uses only past information: `df['Correct_Feature'] = df['High'].shift(1) - df['Close'].shift(1)`. This uses yesterday's high minus yesterday's close, information fully available before today's market open. The correlation drops to a realistic ~0.15, but this is a tradable signal that can be implemented in live trading. The example starkly illustrates why look-ahead bias is called "the killer of trading strategies"—it produces backtests with 90%+ accuracy that fail immediately in live trading.

**Pitfall 2: Data Leakage (Train-Test Contamination)**

Data leakage occurs when information from the test set influences the training process, leading to optimistic performance estimates that don't generalize. The `demonstrate_data_leakage()` method focuses on a common leakage source in time-series: scaling and normalization.

The **wrong approach** fits a `StandardScaler` on the entire dataset (train + test) before splitting: `scaler_wrong.fit_transform(X_full)`. This means the training data is scaled using statistics (mean, standard deviation) that include information from the test set. If the test period contains a market crash (e.g., NEPSE dropping 20% in a month), the mean used to scale the training data includes this crash information, effectively leaking future market conditions into the training set.

The **correct approach** fits the scaler **only** on the training data: `scaler_correct.fit_transform(X_train_raw)`, then applies the same transformation to the test set: `scaler_correct.transform(X_test_raw)`. This ensures the model trains on data scaled by training-period statistics only, and the test set is transformed by the same parameters, simulating real deployment where future statistics are unknown. This is particularly critical for NEPSE, where volatility regimes can shift dramatically during political events or regulatory changes.

**Pitfall 3: Multicollinearity (Redundant Features)**

Multicollinearity occurs when features are highly correlated with each other, providing redundant information. While not strictly a "leakage" issue, it creates instability in linear models (wildly varying coefficients) and wastes computational resources. The `demonstrate_multicollinearity()` method shows this with moving averages.

The example creates `Close_SMA_5`, `Close_SMA_5_Lag1`, and `Close_SMA_5_Lag2`—three features that are nearly identical (correlation > 0.95). A 5-day moving average shifted by 1 day is almost the same as the original 5-day average, just missing one day and adding another. Keeping all three adds no new information but triples the noise and computational load.

The **solution** uses a correlation threshold (0.95) to identify and remove highly correlated features. When two features correlate above this threshold, the code removes the one with lower individual correlation to the target (Close price). This preserves the most predictive member of each correlated group while ensuring feature diversity. For NEPSE, this might mean keeping `EMA_20` but dropping `SMA_20` if they are highly correlated, or keeping volume-based features but dropping price-based ones that convey the same trend information.

**The Critical Distinction:**
The implementation emphasizes that **Feature Engineering** is about **recall**—ensuring no predictive signal is missed by creating a comprehensive feature set. **Feature Selection** is about **precision**—ensuring only reliable, non-redundant signals are used by filtering out noise and multicollinearity. For the NEPSE prediction system, this two-phase approach ensures the model captures the complex temporal dynamics of the Nepalese market—from fiscal year seasonality to circuit breaker effects—while maintaining the statistical rigor necessary for reliable forecasting in an emerging market context.

---

## **10.9 Feature Documentation**

Feature documentation is the systematic recording of metadata, definitions, transformations, and business logic for every feature used in the NEPSE prediction system. Proper documentation ensures reproducibility, facilitates team collaboration, enables regulatory compliance, and preserves institutional knowledge about why specific features were engineered and how they should be interpreted.

For production machine learning systems, undocumented features become "technical debt"—when the original engineer leaves, no one understands why `Feature_X` was created or how it was calculated, leading to maintenance nightmares and model degradation. In regulated financial environments, documentation is often legally required to explain model decisions to auditors and regulators.

```python
import pandas as pd
import numpy as np
import json
from datetime import datetime
from typing import Dict, List, Any
import hashlib

class NEPSEFeatureDocumenter:
    """
    Comprehensive feature documentation system for NEPSE stock prediction.
    Implements feature versioning, lineage tracking, and metadata management.
    """
    
    def __init__(self, feature_df: pd.DataFrame, target_col: str = 'Close'):
        self.feature_df = feature_df
        self.target_col = target_col
        self.documentation = {
            'system': 'NEPSE Stock Prediction System',
            'version': '1.0.0',
            'created_at': datetime.now().isoformat(),
            'total_records': len(feature_df),
            'features': {}
        }
        
    def document_raw_features(self):
        """Document raw input features from NEPSE CSV"""
        raw_features = {
            'Open': {
                'description': 'Opening price of the trading day',
                'type': 'numeric',
                'source': 'NEPSE CSV column "Open"',
                'units': 'NPR (Nepalese Rupees)',
                'temporal_validity': 'Known at market open (11:00 AM NPT)',
                'limitations': 'May gap from previous close due to overnight news'
            },
            'High': {
                'description': 'Highest price traded during the day',
                'type': 'numeric', 
                'source': 'NEPSE CSV column "High"',
                'units': 'NPR',
                'temporal_validity': 'Only known after market close (3:00 PM NPT)',
                'caution': 'USING THIS FOR SAME-DAY PREDiction = LOOK-AHEAD BIAS',
                'valid_uses': 'Calculate range for next-day features only'
            },
            'Low': {
                'description': 'Lowest price traded during the day',
                'type': 'numeric',
                'source': 'NEPSE CSV column "Low"',
                'units': 'NPR',
                'temporal_validity': 'Only known after market close',
                'caution': 'Same look-ahead bias risk as High'
            },
            'Close': {
                'description': 'Closing price of the trading day',
                'type': 'numeric',
                'source': 'NEPSE CSV column "Close"',
                'units': 'NPR',
                'temporal_validity': 'Known after market close (3:00 PM NPT)',
                'target_variable': True,
                'prediction_horizon': 'Next day close price',
                'usage': 'Target variable for supervised learning'
            },
            'Vol': {
                'description': 'Trading volume (number of shares traded)',
                'type': 'numeric',
                'source': 'NEPSE CSV column "Vol"',
                'units': 'Shares',
                'temporal_validity': 'Known after market close',
                'market_context': 'NEPSE has relatively low liquidity; volume spikes significant',
                'interpretation': 'High volume + price change = strong conviction'
            },
            'Prev. Close': {
                'description': 'Previous trading day closing price',
                'type': 'numeric',
                'source': 'NEPSE CSV column "Prev. Close"',
                'units': 'NPR',
                'temporal_validity': 'Known at market open (valid for prediction)',
                'usage': 'Calculate daily returns, gap analysis'
            }
        }
        
        self.documentation['features'].update(raw_features)
        print("Documented 7 raw input features from NEPSE CSV")
        return self
    
    def document_engineered_features(self):
        """Document derived features with transformation logic"""
        engineered = {
            'Daily_Return': {
                'description': 'Percentage change from previous close',
                'formula': '((Close - Prev. Close) / Prev. Close) * 100',
                'type': 'derived',
                'parent_features': ['Close', 'Prev. Close'],
                'transformation': 'Percentage change',
                'stationarity': 'More stationary than raw prices',
                'interpretation': 'Daily profit/loss percentage',
                'usage': 'Target variable for return prediction models'
            },
            'Close_Lag_1': {
                'description': 'Previous day closing price (1-period lag)',
                'formula': 'Close.shift(1)',
                'type': 'lag',
                'parent_features': ['Close'],
                'transformation': 'Temporal shift (1 day)',
                'temporal_validity': 'Available at prediction time (t-1)',
                'caution': 'Must use shift(1), not current value',
                'autocorrelation': 'High with Close (momentum effect)'
            },
            'SMA_20': {
                'description': '20-day Simple Moving Average of Close price',
                'formula': 'Close.rolling(window=20).mean()',
                'type': 'rolling',
                'parent_features': ['Close'],
                'transformation': 'Moving average (20 periods)',
                'lookback': '20 trading days (~1 month in NEPSE)',
                'interpretation': 'Short-term trend indicator',
                'lag': 'Lags behind price by ~10 days (centered)',
                'usage': 'Trend following, support/resistance level'
            },
            'RSI': {
                'description': 'Relative Strength Index (14-period)',
                'formula': '100 - (100 / (1 + RS)), where RS = avg_gain / avg_loss',
                'type': 'technical_indicator',
                'parent_features': ['Close'],
                'transformation': 'Momentum oscillator (0-100 scale)',
                'parameters': {'period': 14, 'overbought': 70, 'oversold': 30},
                'interpretation': 'Momentum and mean-reversion indicator',
                'temporal_validity': 'Uses 14 days of history, no lookahead',
                'caution': 'Can stay overbought/oversold in strong trends'
            },
            'Volume_Z_Score': {
                'description': 'Standardized volume (anomaly detection)',
                'formula': '(Volume - Volume.mean(20)) / Volume.std(20)',
                'type': 'statistical',
                'parent_features': ['Vol'],
                'transformation': 'Z-score normalization (20-day window)',
                'interpretation': 'Unusual volume activity (>2 = anomaly)',
                'market_context': 'NEPSE liquidity alerts',
                'usage': 'Detect institutional accumulation/distribution'
            }
        }
        
        self.documentation['features'].update(engineered)
        print(f"Documented {len(engineered)} engineered features")
        return self
    
    def create_feature_lineage(self):
        """
        Create lineage tracking showing how features are derived.
        Essential for debugging and regulatory compliance.
        """
        lineage = {
            'raw_data_sources': {
                'nepse_csv': {
                    'path': 'data/nepse_ohlcv.csv',
                    'columns': ['Open', 'High', 'Low', 'Close', 'Vol', 'Prev. Close'],
                    'frequency': 'daily',
                    'market': 'NEPSE'
                }
            },
            'feature_transformations': [
                {
                    'step': 1,
                    'operation': 'lag_creation',
                    'input': ['Close'],
                    'output': ['Close_Lag_1', 'Close_Lag_3', 'Close_Lag_5'],
                    'parameters': {'lags': [1, 3, 5]},
                    'timestamp': datetime.now().isoformat()
                },
                {
                    'step': 2,
                    'operation': 'rolling_statistics',
                    'input': ['Close'],
                    'output': ['SMA_20', 'EMA_20', 'Volatility_20'],
                    'parameters': {'window': 20, 'functions': ['mean', 'ewm_mean', 'std']},
                    'timestamp': datetime.now().isoformat()
                },
                {
                    'step': 3,
                    'operation': 'technical_indicators',
                    'input': ['Close', 'High', 'Low'],
                    'output': ['RSI', 'MACD', 'BB_Upper', 'BB_Lower'],
                    'parameters': {'rsi_period': 14, 'macd_fast': 12, 'macd_slow': 26},
                    'timestamp': datetime.now().isoformat()
                }
            ],
            'validation_checks': {
                'look_ahead_bias': 'Verified: All features use shift() for temporal alignment',
                'stationarity': 'ADF test passed for return-based features',
                'multicollinearity': 'Correlation matrix checked, VIF < 5 for all selected',
                'data_leakage': 'Train-test split maintained throughout pipeline'
            }
        }
        
        self.documentation['lineage'] = lineage
        print("\nFeature lineage documentation created")
        print(f"  - {len(lineage['feature_transformations'])} transformation steps documented")
        print(f"  - Validation checks recorded")
        return self
    
    def export_documentation(self, filepath='nepse_feature_documentation.json'):
        """Export complete documentation to JSON"""
        with open(filepath, 'w') as f:
            json.dump(self.documentation, f, indent=2, default=str)
        print(f"\n✓ Feature documentation exported to {filepath}")
        return self

# Usage example
if __name__ == "__main__":
    # Create sample data
    np.random.seed(42)
    dates = pd.date_range('2023-01-01', periods=200, freq='B')
    prices = 1000 + np.cumsum(np.random.randn(200) * 10)
    
    df = pd.DataFrame({
        'Date': dates,
        'Open': prices + np.random.randn(200) * 5,
        'High': prices + abs(np.random.randn(200)) * 8,
        'Low': prices - abs(np.random.randn(200)) * 8,
        'Close': prices,
        'Vol': np.random.lognormal(12, 0.8, 200),
        'Prev. Close': [np.nan] + list(prices[:-1])
    })
    
    # Initialize documenter
    doc = NEPSEFeatureDocumenter(df)
    
    # Create comprehensive documentation
    (doc
     .document_raw_features()
     .document_engineered_features()
     .create_feature_lineage()
     .export_documentation('nepse_features_v1.json'))
    
    # Display sample documentation
    print("\n" + "="*70)
    print("SAMPLE FEATURE DOCUMENTATION")
    print("="*70)
    
    # Show documentation for a critical feature
    if 'High' in doc.documentation['features']:
        high_doc = doc.documentation['features']['High']
        print(f"\nFeature: High")
        print(f"Description: {high_doc['description']}")
        print(f"Temporal Validity: {high_doc['temporal_validity']}")
        print(f"⚠️  Caution: {high_doc.get('caution', 'None')}")
    
    if 'Daily_Return' in doc.documentation['features']:
        return_doc = doc.documentation['features']['Daily_Return']
        print(f"\nFeature: Daily_Return")
        print(f"Formula: {return_doc['formula']}")
        print(f"Parent Features: {', '.join(return_doc['parent_features'])}")
        print(f"Interpretation: {return_doc['interpretation']}")

```

**Explanation:**

This implementation demonstrates a comprehensive feature documentation system essential for production machine learning systems, particularly in regulated financial environments like the NEPSE prediction system.

**The Documentation Structure:**
The `NEPSEFeatureDocumenter` class creates a structured metadata repository that tracks every aspect of feature creation and usage. Unlike informal code comments, this system creates machine-readable documentation that can be versioned, audited, and automatically validated.

**Raw Feature Documentation:**
The `document_raw_features()` method records metadata for each column in the original NEPSE CSV file. For the `High` feature, it explicitly documents the **temporal validity**—this value is only known after market close (3:00 PM NPT), making it unsuitable for same-day prediction features without proper lagging. This documentation prevents the common look-ahead bias error where engineers accidentally use same-day highs to predict same-day closes. The `caution` field specifically warns: "USING THIS FOR SAME-DAY PREDICTION = LOOK-AHEAD BIAS"—a critical safety measure for team members who might not be familiar with financial data constraints.

**Engineered Feature Documentation:**
For derived features like `Daily_Return`, the documentation captures the **transformation formula** (`((Close - Prev. Close) / Prev. Close) * 100`), **parent features** (dependencies), and **stationarity properties**. This is crucial for the NEPSE system because it explains why we use returns instead of raw prices—returns are more stationary and comparable across different stocks and time periods. The documentation also notes that `Close_Lag_1` must use `shift(1)` rather than current values, preserving temporal integrity.

**Feature Lineage Tracking:**
The `create_feature_lineage()` method implements **data lineage**—a complete audit trail showing how raw NEPSE CSV data transforms into model-ready features. This tracks:
- **Provenance**: Raw data sources (`nepse_csv`) with file paths and schema
- **Transformations**: Step-by-step operations (lag creation → rolling statistics → technical indicators) with timestamps
- **Parameters**: Configuration details (e.g., RSI period = 14 days) that affect feature calculation
- **Validation Checks**: Verification that look-ahead bias was avoided, stationarity was checked, and multicollinearity was controlled

This lineage is essential for **regulatory compliance** in financial markets. If the Nepal Securities Board (SEBON) audits the prediction system, this documentation proves that the model doesn't use future information or insider data. It's also critical for **model debugging**—if the `RSI` feature starts producing unexpected values, the lineage trace shows it depends on 14 days of `Close` prices, helping engineers quickly identify if the input data has gaps or errors.

**Version Control and Reproducibility:**
The `export_documentation()` method creates a JSON artifact (`nepse_features_v1.json`) that can be versioned alongside code. When the feature engineering pipeline is updated (e.g., adding new technical indicators), the version number increments, creating a clear history of changes. This ensures that experiments are reproducible—if a model trained on January 1st achieves 85% accuracy, the documentation allows engineers to reconstruct the exact feature set used, even if the pipeline has evolved since then.

**Practical Safety Mechanisms:**
The documentation system acts as a safety check by explicitly flagging risky features. For example, `High` and `Low` are marked with temporal validity warnings because they represent intraday extremes unknown until market close. This prevents junior team members from accidentally creating features like `(High - Open) / Open` as a predictor of `Close` for the same day—a feature that would show 95% accuracy in backtests but be impossible to calculate in live trading (since you don't know the High until the day is over).

For the NEPSE prediction system, this documentation infrastructure ensures that the transition from research to production maintains data integrity, complies with financial regulations, and preserves institutional knowledge about feature construction logic.

---

## **Chapter Summary**

In this chapter, we established the foundational principles of feature engineering for time-series prediction systems, using the NEPSE (Nepal Stock Exchange) stock prediction system as our primary example. We covered the entire lifecycle from raw OHLCV data to model-ready feature matrices.

### **Key Concepts Mastered:**

**1. Feature Fundamentals (10.1)**
We learned that features are measurable representations of the underlying phenomenon. For NEPSE data, raw columns like `Open`, `High`, `Low`, `Close`, and `Vol` serve as basic features, but derived features like `Daily_Return` (percentage change) often provide more predictive power because they normalize price levels and capture momentum rather than absolute values.

**2. The Feature Engineering Process (10.2)**
We implemented a structured pipeline with five stages: **Exploratory Data Analysis** (understanding NEPSE's Sunday-Thursday trading pattern and circuit breaker regulations), **Feature Ideation** (planning features based on domain knowledge), **Feature Construction** (computing lag, rolling, and technical indicators), **Feature Transformation** (scaling and normalization), and **Feature Validation** (checking for stationarity and data leakage). This systematic approach ensures reproducibility and quality.

**3. Time-Series Feature Types (10.3)**
We explored specialized feature categories essential for financial prediction:
- **Lag Features**: Autoregressive components (`Close_Lag_1`, `Volume_Lag_3`) that capture temporal dependencies and momentum
- **Rolling Window Features**: Local statistics (`SMA_20`, `Volatility_20`) that adapt to changing market regimes and measure trend/volatility
- **Expanding Window Features**: Cumulative statistics (`Exp_Mean`, `Cumulative_Return`) that provide long-term context
- **DateTime Features**: Calendar effects specific to NEPSE's fiscal year (mid-July ending) and Sunday-Thursday trading schedule

**4. Domain Knowledge Integration (10.4)**
We integrated NEPSE-specific domain knowledge including:
- **Fiscal Calendar Features**: Nepal's fiscal year (Shrawan to Ashad) creates distinct seasonal patterns in Q4 (April-July) due to year-end portfolio adjustments
- **Circuit Breaker Proximity**: Features measuring distance to 4% daily limits, capturing volatility regimes unique to NEPSE's regulatory structure
- **Trading Calendar Adjustments**: Handling Friday-Saturday weekends (non-trading days) and the liquidity constraints of an emerging market

**5. Feature Quality Assessment (10.5)**
We implemented statistical validation including:
- **Stationarity Testing**: Augmented Dickey-Fuller tests to check if price series are mean-reverting (required for many time-series models)
- **Look-Ahead Bias Detection**: Verification that lag features properly use `shift()` operations to prevent future information leakage
- **Multicollinearity Analysis**: Correlation matrices and VIF scores to remove redundant features (e.g., highly correlated moving averages)

**6. Engineering vs. Selection (10.6)**
We distinguished between **Feature Engineering** (the creative expansion phase generating 100+ features to maximize information capture) and **Feature Selection** (the analytical reduction phase using Variance Threshold, Correlation Analysis, Mutual Information, and Lasso regularization to prevent overfitting). The NEPSE system first engineers comprehensive features (lags, technicals, domain-specific), then selects the top 15-20 robust predictors.

**7. Automation (10.7)**
We explored automated feature engineering using `tsfresh` (extracting 800+ statistical features automatically) and `featuretools` (deep feature synthesis). While automation efficiently generates candidates, we emphasized that domain knowledge remains essential for NEPSE-specific features (fiscal calendars, circuit breakers) that generic libraries cannot infer.

**8. Common Pitfalls (10.8)**
We demonstrated critical errors specific to financial time-series:
- **Look-Ahead Bias**: Using same-day `High` to predict `Close` creates unrealistic 85% correlations that fail in live trading
- **Data Leakage**: Scaling on full datasets (train + test) contaminates training with test statistics, causing optimistic backtests
- **Multicollinearity**: Keeping highly correlated features (ρ > 0.95) destabilizes linear models without adding information

**9. Documentation (10.9)**
We established a comprehensive documentation system tracking feature lineage, transformation formulas, temporal validity constraints, and validation checks. This ensures regulatory compliance, prevents look-ahead bias through explicit warnings, and maintains reproducibility across team members and system versions.

### **Practical Skills Acquired:**

- **Implementation**: Building end-to-end feature pipelines for CSV-based NEPSE data
- **Validation**: Detecting and preventing data leakage in temporal data
- **Domain Adaptation**: Engineering features specific to emerging market characteristics (low liquidity, circuit breakers, unique fiscal calendars)
- **Quality Assurance**: Statistical testing for stationarity, multicollinearity, and predictive power
- **Production Readiness**: Documenting features for regulatory compliance and team collaboration

### **Next Steps:**

In **Chapter 11: Basic Feature Creation**, we will dive deeper into specific feature construction techniques, implementing lag features, rolling statistics, percentage changes, and time-based encodings with production-ready code patterns optimized for the NEPSE prediction system. We will move from the conceptual framework established here to concrete implementation details and performance optimization strategies.

---

**End of Chapter 10**